In [1]:
import numpy as np
import pandas as pd
import os
import math
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from pykakasi import kakasi

from typing import Optional, Tuple, List
from tqdm.notebook import tqdm
import argparse

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [676]:
torch.set_printoptions(threshold=100)

![](image/transformer.png)
参照: https://qiita.com/FuwaraMiyasaki/items/239f3528053889847825

# 入力

In [128]:
max_len = 10
d_model = 8
batch_size = 3
n_heads = 2
n_layers = 5
pad = '<pad>'  # 特殊文字
unk = '<unk>'  # 特殊文字

word_list = [
    ['私', 'は', 'AI', 'です', '。', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'],
    ['私', 'が', 'AI', 'だ', '。', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'],
    ['私', 'の', 'AI', 'は', '最強', 'です', '。', '<pad>', '<pad>', '<pad>']
]

In [162]:
# デバッグ用
max_len = 5
d_model = 4
batch_size = 2
n_heads = 2
n_layers = 2
pad = '<pad>'  # 特殊文字
unk = '<unk>'  # 特殊文字

word_list = [
    ['私', 'は', 'AI', 'です', '<pad>'],
    ['あなた', 'は', '最強', 'です', '<pad>']
]

## embedding

In [163]:
le = preprocessing.LabelEncoder()
le.fit(sum(word_list, []))
ids_list = torch.tensor([le.transform(w) for w in word_list])
ids_list

tensor([[6, 4, 1, 3, 0],
        [2, 4, 5, 3, 0]])

In [164]:
token_embeding = nn.Embedding(
    num_embeddings=len(le.classes_) + 1,  # +1の理由:特殊文字
    embedding_dim=d_model
)

embed_token = token_embeding(ids_list)
embed_token, embed_token.shape

(tensor([[[ 1.3975e+00, -4.6678e-01,  5.6274e-02, -6.3892e-01],
          [-5.5102e-01,  6.1614e-01,  1.5933e+00, -1.1261e+00],
          [-8.5371e-01, -3.8980e-01, -1.2629e+00,  7.9844e-01],
          [ 4.0651e-02,  6.5141e-01, -1.7298e+00, -3.5171e-01],
          [-5.3642e-01, -1.0908e+00,  2.0907e-01,  5.6855e-01]],
 
         [[-3.7238e-01, -6.4494e-01, -3.1476e-01,  3.5265e-01],
          [-5.5102e-01,  6.1614e-01,  1.5933e+00, -1.1261e+00],
          [ 7.6704e-01,  4.7227e-02,  4.1925e-04, -6.3587e-01],
          [ 4.0651e-02,  6.5141e-01, -1.7298e+00, -3.5171e-01],
          [-5.3642e-01, -1.0908e+00,  2.0907e-01,  5.6855e-01]]],
        grad_fn=<EmbeddingBackward>),
 torch.Size([2, 5, 4]))

```python
torch.Size[バッチサイズ, 文章の最大長, 単語ベクトル次元]
```

## Positional Encoding

以下のような関数を作成して都度計算するのではなく、表として残しておくために一気に計算しておく

In [165]:
pos = torch.arange(0, max_len).unsqueeze(1)
print(f'pos: {pos}')
pe_denominator = torch.exp(
    torch.arange(0, d_model, 2) * math.log(10000) / d_model
)
print(f'pos / pe_denominator: {pos / pe_denominator}')

pe = torch.zeros(max_len, d_model)
pe[:, 0::2] = torch.sin(pos / pe_denominator)  # 偶数項にはsin
pe[:, 1::2] = torch.cos(pos / pe_denominator)  # 奇数項にはcos
print(f'pe: {pe}')

pe = pe.unsqueeze(0)  # バッチに対応するため
print(f'pe: {pe}')

pos: tensor([[0],
        [1],
        [2],
        [3],
        [4]])
pos / pe_denominator: tensor([[0.0000, 0.0000],
        [1.0000, 0.0100],
        [2.0000, 0.0200],
        [3.0000, 0.0300],
        [4.0000, 0.0400]])
pe: tensor([[ 0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0100,  0.9999],
        [ 0.9093, -0.4161,  0.0200,  0.9998],
        [ 0.1411, -0.9900,  0.0300,  0.9996],
        [-0.7568, -0.6536,  0.0400,  0.9992]])
pe: tensor([[[ 0.0000,  1.0000,  0.0000,  1.0000],
         [ 0.8415,  0.5403,  0.0100,  0.9999],
         [ 0.9093, -0.4161,  0.0200,  0.9998],
         [ 0.1411, -0.9900,  0.0300,  0.9996],
         [-0.7568, -0.6536,  0.0400,  0.9992]]])


![](Image/positional_encoding.png)

- pos : 単語の位置
- i : 成分の次元

### クラス化

In [166]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout_p: float, max_len: int):
        super(PositionalEncoding, self).__init__()
        
        self.dropout = nn.Dropout(p=dropout_p)
        # Not a parameter
        self.register_buffer('pe', self._pe_table(d_model, max_len))

    def _pe_table(self, d_model: int, max_len: int) -> torch.Tensor:
        pos = torch.arange(0, max_len).unsqueeze(1)
        pe_denominator = torch.exp(
            torch.arange(0, d_model, 2) * math.log(10000) / d_model
        )

        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(pos / pe_denominator)  # 偶数項にはsin
        pe[:, 1::2] = torch.cos(pos / pe_denominator)  # 奇数項にはcos

        pe = pe.unsqueeze(0)  # バッチに対応するため
        return pe

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.dropout(x + self.pe[:, :x.size(1)].clone().detach())

In [167]:
positional_encoder = PositionalEncoding(
    d_model=d_model, dropout_p=0.0, max_len=max_len)

positional_token = positional_encoder(embed_token)
positional_token, positional_token.shape

(tensor([[[ 1.3975,  0.5332,  0.0563,  0.3611],
          [ 0.2905,  1.1564,  1.6033, -0.1261],
          [ 0.0556, -0.8059, -1.2429,  1.7982],
          [ 0.1818, -0.3386, -1.6998,  0.6478],
          [-1.2932, -1.7445,  0.2491,  1.5678]],
 
         [[-0.3724,  0.3551, -0.3148,  1.3527],
          [ 0.2905,  1.1564,  1.6033, -0.1261],
          [ 1.6763, -0.3689,  0.0204,  0.3639],
          [ 0.1818, -0.3386, -1.6998,  0.6478],
          [-1.2932, -1.7445,  0.2491,  1.5678]]], grad_fn=<AddBackward0>),
 torch.Size([2, 5, 4]))

```python
torch.Size[バッチサイズ, 文章の最大長, 単語ベクトル次元]
```

In [168]:
# デバッグ用
positional_token = torch.tensor(
    [[[1., 1., 1., 1.],
      [2., 2., 2., 2.],
      [3., 3., 3., 3.],
      [4., 4., 4., 4.],
      [5., 5., 5., 5.]],
 
     [[0., 0., 0., 0.],
      [1., 1., 1., 1.],
      [2., 2., 2., 2.],
      [3., 3., 3., 3.],
      [4., 4., 4., 4.]]]
)

# ネットワーク

## Scaled Dot-Product Attention

![](image/scaled_dot-product_attention.png)
参照: Attention is all you need.

### self-attention

In [169]:
query = positional_token
key = positional_token
value = positional_token

In [174]:
W_q = nn.Linear(in_features=d_model,
                out_features=d_model,
                bias=False)
W_k = nn.Linear(in_features=d_model,
                out_features=d_model,
                bias=False)
W_v = nn.Linear(in_features=d_model,
                out_features=d_model,
                bias=False)

In [175]:
W_q.weight.data

tensor([[-0.3782,  0.0654, -0.4321,  0.2999],
        [ 0.2104,  0.4932, -0.4274, -0.4027],
        [ 0.4142, -0.4972,  0.1328,  0.4964],
        [-0.1588, -0.0803, -0.2237, -0.2724]])

In [176]:
# デバッグ用
W_q.weight.data = torch.tensor([[0.5, 0.5, 0.5, 0.5],
                                [0.5, 0.5, 0.5, 0.5],
                                [0.5, 0.5, 0.5, 0.5],
                                [0.5, 0.5, 0.5, 0.5]])
W_k.weight.data = torch.tensor([[0.5, 0.5, 0.5, 0.5],
                                [0.5, 0.5, 0.5, 0.5],
                                [0.5, 0.5, 0.5, 0.5],
                                [0.5, 0.5, 0.5, 0.5]])
W_v.weight.data = torch.tensor([[0.5, 0.5, 0.5, 0.5],
                                [0.5, 0.5, 0.5, 0.5],
                                [0.5, 0.5, 0.5, 0.5],
                                [0.5, 0.5, 0.5, 0.5]])

In [177]:
Q_single = W_q(query)
K_single = W_k(key)
V_single = W_v(value)

In [178]:
Q_single, Q_single.shape

(tensor([[[ 2.,  2.,  2.,  2.],
          [ 4.,  4.,  4.,  4.],
          [ 6.,  6.,  6.,  6.],
          [ 8.,  8.,  8.,  8.],
          [10., 10., 10., 10.]],
 
         [[ 0.,  0.,  0.,  0.],
          [ 2.,  2.,  2.,  2.],
          [ 4.,  4.,  4.,  4.],
          [ 6.,  6.,  6.,  6.],
          [ 8.,  8.,  8.,  8.]]], grad_fn=<UnsafeViewBackward>),
 torch.Size([2, 5, 4]))

文章と文章の単語間の関係を表している
```python
torch.Size([バッチサイズ, 文章の最大長, 文章の最大長])
```

上記で、一括して、queryと(W_0, W_1, ... , W_head)との内積を求めたが、 multi heads attentionをするには、以下のようにQ,K,Vをヘッドで分割する必要がある
```python
torch.Size([バッチサイズ, ヘッド, 文章の最大長, 単語のベクトル次元])
```

In [179]:
print(f'single head Q:\n{Q_single}')
print(f'{Q_single.shape}')
Q_multi = Q_single.contiguous().view(batch_size, 
                                     -1, 
                                     n_heads,
                                     d_model//n_heads).transpose(1,2)
print(f'multi heads Q :\n{Q_multi}')
print(f'{Q_multi.shape}')

single head Q:
tensor([[[ 2.,  2.,  2.,  2.],
         [ 4.,  4.,  4.,  4.],
         [ 6.,  6.,  6.,  6.],
         [ 8.,  8.,  8.,  8.],
         [10., 10., 10., 10.]],

        [[ 0.,  0.,  0.,  0.],
         [ 2.,  2.,  2.,  2.],
         [ 4.,  4.,  4.,  4.],
         [ 6.,  6.,  6.,  6.],
         [ 8.,  8.,  8.,  8.]]], grad_fn=<UnsafeViewBackward>)
torch.Size([2, 5, 4])
multi heads Q :
tensor([[[[ 2.,  2.],
          [ 4.,  4.],
          [ 6.,  6.],
          [ 8.,  8.],
          [10., 10.]],

         [[ 2.,  2.],
          [ 4.,  4.],
          [ 6.,  6.],
          [ 8.,  8.],
          [10., 10.]]],


        [[[ 0.,  0.],
          [ 2.,  2.],
          [ 4.,  4.],
          [ 6.,  6.],
          [ 8.,  8.]],

         [[ 0.,  0.],
          [ 2.,  2.],
          [ 4.,  4.],
          [ 6.,  6.],
          [ 8.,  8.]]]], grad_fn=<TransposeBackward0>)
torch.Size([2, 2, 5, 2])


In [180]:
K_multi = K_single.contiguous().view(batch_size, 
                                     -1, 
                                     n_heads,
                                     d_model//n_heads).transpose(1,2)
V_multi = V_single.contiguous().view(batch_size, 
                                     -1, 
                                     n_heads,
                                     d_model//n_heads).transpose(1,2)

In [181]:
attn = torch.matmul(Q_multi, K_multi.transpose(2, 3))
attn, attn.shape

(tensor([[[[  8.,  16.,  24.,  32.,  40.],
           [ 16.,  32.,  48.,  64.,  80.],
           [ 24.,  48.,  72.,  96., 120.],
           [ 32.,  64.,  96., 128., 160.],
           [ 40.,  80., 120., 160., 200.]],
 
          [[  8.,  16.,  24.,  32.,  40.],
           [ 16.,  32.,  48.,  64.,  80.],
           [ 24.,  48.,  72.,  96., 120.],
           [ 32.,  64.,  96., 128., 160.],
           [ 40.,  80., 120., 160., 200.]]],
 
 
         [[[  0.,   0.,   0.,   0.,   0.],
           [  0.,   8.,  16.,  24.,  32.],
           [  0.,  16.,  32.,  48.,  64.],
           [  0.,  24.,  48.,  72.,  96.],
           [  0.,  32.,  64.,  96., 128.]],
 
          [[  0.,   0.,   0.,   0.,   0.],
           [  0.,   8.,  16.,  24.,  32.],
           [  0.,  16.,  32.,  48.,  64.],
           [  0.,  24.,  48.,  72.,  96.],
           [  0.,  32.,  64.,  96., 128.]]]], grad_fn=<UnsafeViewBackward>),
 torch.Size([2, 2, 5, 5]))

```python
torch.Size([バッチサイズ, ヘッド, 文章の最大長, 文章の最大長])
```
headごとの単語の関係が表される

#### pad mask

Masked Multi-Head Attention用にmaskを用意する

[ここ](https://qiita.com/FuwaraMiyasaki/items/239f3528053889847825#全体図:~:text=Attention%E3%81%AEMasking%E3%81%AE%E5%AE%9F%E8%A3%85%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6,-Attention%E3%81%AE%E3%83%9E%E3%82%B9%E3%82%AF%E3%81%AE%E5%AE%9F%E8%A3%85%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6%E6%82%A9%E3%82%93%E3%81%A0%E3%81%AE%E3%81%A7%E3%80%81Harvard)を参照した

**入力文**のpadの部分を無視するようなmaskをする(参照される方のpadは無視していない)

|\\|   参照文    |    |    |    |  |    |  |  |  |  |  | 
|:---:| :---: | :--: | :--: | :--: | :--: | :--: | :---: | :---: | :---: | :---: | :---: | 
|入力文|    \\   | 私   | は   | AI   | です | 。   | \<pad> | \<pad> | \<pad> | \<pad> | \<pad> | 
|| 私    | True | True | True | True | True | False | False | False | False | False | 
|| は    | True | True | True | True | True | False | False | False | False | False | 
|| AI    | True | True | True | True | True | False | False | False | False | False | 
|| です  | True | True | True | True | True | False | False | False | False | False | 
|| 。    | True | True | True | True | True | False | False | False | False | False | 
|| \<pad> | **True** | **True** | **True** | **True** | **True** | False | False | False | False | False | 
|| \<pad> | **True** | **True** | **True** | **True** | **True** | False | False | False | False | False | 
|| \<pad> | **True** | **True** | **True** | **True** | **True** | False | False | False | False | False | 
|| \<pad> | **True** | **True** | **True** | **True** | **True** | False | False | False | False | False | 
|| \<pad> | **True** | **True** | **True** | **True** | **True** | False | False | False | False | False | 

In [182]:
pad_idx = le.transform([pad])[0]
pad_idx

0

In [183]:
ids_list

tensor([[6, 4, 1, 3, 0],
        [2, 4, 5, 3, 0]])

In [184]:
pad_mask = (ids_list != pad_idx)
pad_mask, pad_mask.shape

(tensor([[ True,  True,  True,  True, False],
         [ True,  True,  True,  True, False]]),
 torch.Size([2, 5]))

`attn`にpad_maskを適用するため、shapeを変形する

In [185]:
(ids_list != pad_idx).unsqueeze(-2).shape

torch.Size([2, 1, 5])

In [186]:
(ids_list != pad_idx).unsqueeze(-2).unsqueeze(1).shape

torch.Size([2, 1, 1, 5])

In [187]:
(ids_list != pad_idx).unsqueeze(-2).unsqueeze(1)

tensor([[[[ True,  True,  True,  True, False]]],


        [[[ True,  True,  True,  True, False]]]])

In [188]:
pad_mask = (ids_list != pad_idx).unsqueeze(-2).unsqueeze(1).expand(attn.shape)
pad_mask, pad_mask.shape

(tensor([[[[ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False]],
 
          [[ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False]]],
 
 
         [[[ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False]],
 
          [[ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True, False]]]]),
 torch.Size([2, 2, 5, 5]

In [189]:
attn = attn.masked_fill(pad_mask == False, -1e9)
attn

tensor([[[[ 8.0000e+00,  1.6000e+01,  2.4000e+01,  3.2000e+01, -1.0000e+09],
          [ 1.6000e+01,  3.2000e+01,  4.8000e+01,  6.4000e+01, -1.0000e+09],
          [ 2.4000e+01,  4.8000e+01,  7.2000e+01,  9.6000e+01, -1.0000e+09],
          [ 3.2000e+01,  6.4000e+01,  9.6000e+01,  1.2800e+02, -1.0000e+09],
          [ 4.0000e+01,  8.0000e+01,  1.2000e+02,  1.6000e+02, -1.0000e+09]],

         [[ 8.0000e+00,  1.6000e+01,  2.4000e+01,  3.2000e+01, -1.0000e+09],
          [ 1.6000e+01,  3.2000e+01,  4.8000e+01,  6.4000e+01, -1.0000e+09],
          [ 2.4000e+01,  4.8000e+01,  7.2000e+01,  9.6000e+01, -1.0000e+09],
          [ 3.2000e+01,  6.4000e+01,  9.6000e+01,  1.2800e+02, -1.0000e+09],
          [ 4.0000e+01,  8.0000e+01,  1.2000e+02,  1.6000e+02, -1.0000e+09]]],


        [[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+09],
          [ 0.0000e+00,  8.0000e+00,  1.6000e+01,  2.4000e+01, -1.0000e+09],
          [ 0.0000e+00,  1.6000e+01,  3.2000e+01,  4.8000e+01, -1.0000

In [190]:
attn = F.softmax(attn, dim=-1)
attn

tensor([[[[3.7739e-11, 1.1250e-07, 3.3535e-04, 9.9966e-01, 0.0000e+00],
          [1.4252e-21, 1.2664e-14, 1.1254e-07, 1.0000e+00, 0.0000e+00],
          [5.3802e-32, 1.4252e-21, 3.7751e-11, 1.0000e+00, 0.0000e+00],
          [2.0305e-42, 1.6038e-28, 1.2664e-14, 1.0000e+00, 0.0000e+00],
          [0.0000e+00, 1.8049e-35, 4.2484e-18, 1.0000e+00, 0.0000e+00]],

         [[3.7739e-11, 1.1250e-07, 3.3535e-04, 9.9966e-01, 0.0000e+00],
          [1.4252e-21, 1.2664e-14, 1.1254e-07, 1.0000e+00, 0.0000e+00],
          [5.3802e-32, 1.4252e-21, 3.7751e-11, 1.0000e+00, 0.0000e+00],
          [2.0305e-42, 1.6038e-28, 1.2664e-14, 1.0000e+00, 0.0000e+00],
          [0.0000e+00, 1.8049e-35, 4.2484e-18, 1.0000e+00, 0.0000e+00]]],


        [[[2.5000e-01, 2.5000e-01, 2.5000e-01, 2.5000e-01, 0.0000e+00],
          [3.7739e-11, 1.1250e-07, 3.3535e-04, 9.9966e-01, 0.0000e+00],
          [1.4252e-21, 1.2664e-14, 1.1254e-07, 1.0000e+00, 0.0000e+00],
          [5.3802e-32, 1.4252e-21, 3.7751e-11, 1.0000e+00,

In [191]:
torch.matmul(attn, V_multi)

tensor([[[[7.9993, 7.9993],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000]],

         [[7.9993, 7.9993],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000]]],


        [[[3.0000, 3.0000],
          [5.9993, 5.9993],
          [6.0000, 6.0000],
          [6.0000, 6.0000],
          [6.0000, 6.0000]],

         [[3.0000, 3.0000],
          [5.9993, 5.9993],
          [6.0000, 6.0000],
          [6.0000, 6.0000],
          [6.0000, 6.0000]]]], grad_fn=<UnsafeViewBackward>)

#### クラス化

In [198]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, scale: int, drop_p: float):
        super(ScaledDotProductAttention, self).__init__()
        
        self.scale = math.sqrt(scale)
        self.dropout = nn.Dropout(drop_p)
        
    def forward(self, 
                Q: torch.Tensor,  # torch.Size([batch_size, n_heads, seq_length, word_dim]) 
                K: torch.Tensor,  # torch.Size([batch_size, n_heads, seq_length, word_dim])
                V: torch.Tensor,  # torch.Size([batch_size, n_heads, seq_length, word_dim])
                mask:Optional[torch.Tensor] = None  # torch.Size([[batch_size, n_heads, seq_length, word_dim]])
               ) -> torch.Tensor:  
        attn = torch.matmul(Q / self.scale, K.transpose(2,3))
        
        if mask is not None:
            attn = attn.masked_fill(mask == False, -1e9)
            
        attn = self.dropout(F.softmax(attn, dim=-1))
        output = torch.matmul(attn, V)
        
        return output, attn

masked_fillは、ブロードキャストにより、expandしなくてもマスクすることができる

In [202]:
d_k = d_model // n_heads
attention = ScaledDotProductAttention(scale=d_k, drop_p=0.0)
heads, attn = attention(Q_multi, K_multi, V_multi, pad_mask)

In [203]:
print(f'heads:{heads.shape}\n{heads}')

heads:torch.Size([2, 2, 5, 2])
tensor([[[[7.9930, 7.9930],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000]],

         [[7.9930, 7.9930],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000]]],


        [[[3.0000, 3.0000],
          [5.9930, 5.9930],
          [6.0000, 6.0000],
          [6.0000, 6.0000],
          [6.0000, 6.0000]],

         [[3.0000, 3.0000],
          [5.9930, 5.9930],
          [6.0000, 6.0000],
          [6.0000, 6.0000],
          [6.0000, 6.0000]]]], grad_fn=<UnsafeViewBackward>)


In [204]:
print(f'attention score:{attn.shape}\n{attn}')

attention score:torch.Size([2, 2, 5, 5])
tensor([[[[4.2487e-08, 1.2162e-05, 3.4813e-03, 9.9651e-01, 0.0000e+00],
          [1.8178e-15, 1.4895e-10, 1.2204e-05, 9.9999e-01, 0.0000e+00],
          [7.7506e-23, 1.8178e-15, 4.2636e-08, 1.0000e+00, 0.0000e+00],
          [3.3046e-30, 2.2186e-20, 1.4895e-10, 1.0000e+00, 0.0000e+00],
          [1.4089e-37, 2.7077e-25, 5.2035e-13, 1.0000e+00, 0.0000e+00]],

         [[4.2487e-08, 1.2162e-05, 3.4813e-03, 9.9651e-01, 0.0000e+00],
          [1.8178e-15, 1.4895e-10, 1.2204e-05, 9.9999e-01, 0.0000e+00],
          [7.7506e-23, 1.8178e-15, 4.2636e-08, 1.0000e+00, 0.0000e+00],
          [3.3046e-30, 2.2186e-20, 1.4895e-10, 1.0000e+00, 0.0000e+00],
          [1.4089e-37, 2.7077e-25, 5.2035e-13, 1.0000e+00, 0.0000e+00]]],


        [[[2.5000e-01, 2.5000e-01, 2.5000e-01, 2.5000e-01, 0.0000e+00],
          [4.2487e-08, 1.2162e-05, 3.4813e-03, 9.9651e-01, 0.0000e+00],
          [1.8178e-15, 1.4895e-10, 1.2204e-05, 9.9999e-01, 0.0000e+00],
          [7.7506

## Multi-Head Attention

![](./image/multi-head_attention.png)

In [205]:
heads

tensor([[[[7.9930, 7.9930],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000]],

         [[7.9930, 7.9930],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000],
          [8.0000, 8.0000]]],


        [[[3.0000, 3.0000],
          [5.9930, 5.9930],
          [6.0000, 6.0000],
          [6.0000, 6.0000],
          [6.0000, 6.0000]],

         [[3.0000, 3.0000],
          [5.9930, 5.9930],
          [6.0000, 6.0000],
          [6.0000, 6.0000],
          [6.0000, 6.0000]]]], grad_fn=<UnsafeViewBackward>)

multi-headsをまとめて(concat)、single-headに変形する

In [206]:
multi_heads = heads.transpose(1, 2).contiguous().view(batch_size, 
                                                      max_len, 
                                                      d_model)
multi_heads, multi_heads.shape

(tensor([[[7.9930, 7.9930, 7.9930, 7.9930],
          [8.0000, 8.0000, 8.0000, 8.0000],
          [8.0000, 8.0000, 8.0000, 8.0000],
          [8.0000, 8.0000, 8.0000, 8.0000],
          [8.0000, 8.0000, 8.0000, 8.0000]],
 
         [[3.0000, 3.0000, 3.0000, 3.0000],
          [5.9930, 5.9930, 5.9930, 5.9930],
          [6.0000, 6.0000, 6.0000, 6.0000],
          [6.0000, 6.0000, 6.0000, 6.0000],
          [6.0000, 6.0000, 6.0000, 6.0000]]], grad_fn=<ViewBackward>),
 torch.Size([2, 5, 4]))

```python
torch.Size([バッチサイズ, 文章の最大長, 単語のベクトル次元])
```

In [207]:
fc = nn.Linear(in_features=d_model,
               out_features=d_model,
               bias=False)
fc(multi_heads)

tensor([[[-5.4351e+00, -2.2248e-03,  4.4748e+00, -7.5195e+00],
         [-5.4399e+00, -2.2267e-03,  4.4788e+00, -7.5260e+00],
         [-5.4399e+00, -2.2268e-03,  4.4788e+00, -7.5261e+00],
         [-5.4399e+00, -2.2268e-03,  4.4788e+00, -7.5261e+00],
         [-5.4399e+00, -2.2268e-03,  4.4788e+00, -7.5261e+00]],

        [[-2.0400e+00, -8.3512e-04,  1.6795e+00, -2.8223e+00],
         [-4.0751e+00, -1.6680e-03,  3.3552e+00, -5.6379e+00],
         [-4.0799e+00, -1.6701e-03,  3.3591e+00, -5.6445e+00],
         [-4.0799e+00, -1.6702e-03,  3.3591e+00, -5.6445e+00],
         [-4.0799e+00, -1.6702e-03,  3.3591e+00, -5.6445e+00]]],
       grad_fn=<UnsafeViewBackward>)

### クラス化

In [208]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, 
                 n_heads: int, 
                 d_model: int, 
                 max_len: int,
                 dropout_p: float, 
                 dropout_p_attn: float,
                 return_attn: bool = False):
        super(MultiHeadAttention, self).__init__()
        
        self.n_heads = n_heads
        self.d_model = d_model
        self.max_len = max_len
        self.return_attn = return_attn
        
        self.W_q = nn.Linear(in_features=d_model,
                             out_features=d_model,
                             bias=False)
        self.W_k = nn.Linear(in_features=d_model,
                             out_features=d_model,
                             bias=False)
        self.W_v = nn.Linear(in_features=d_model,
                             out_features=d_model,
                             bias=False)
        self.fc = nn.Linear(in_features=d_model,
                            out_features=d_model,
                            bias=False)
        
        d_k = d_model // n_heads
        self.attention = ScaledDotProductAttention(
            scale=d_k, drop_p=dropout_p_attn
        )
        
        self.dropout = nn.Dropout(dropout_p)
        
    def forward(self, 
                query: torch.Tensor,
                key: torch.Tensor,
                value: torch.Tensor,
                mask: Optional[torch.Tensor] = None
               ) -> torch.Tensor:
        batch_size = query.size(0)
        
        Q_single = self.W_q(query)
        K_single = self.W_k(key)
        V_single = self.W_v(value)
        
        Q_multi = Q_single.contiguous().view(
            batch_size, -1, self.n_heads, self.d_model//self.n_heads
        ).transpose(1,2)
        K_multi = K_single.contiguous().view(
            batch_size, -1, self.n_heads, self.d_model//self.n_heads
        ).transpose(1,2)
        V_multi = V_single.contiguous().view(
            batch_size, -1, self.n_heads, self.d_model//self.n_heads
        ).transpose(1,2)
        
        if mask is not None:
            mask = mask.unsqueeze(1)  # heads次元の追加
        
        # Scaled Dot-Product Attention
        heads, attn = self.attention(Q_multi, K_multi, V_multi, mask)
        
        # Concat
        multi_head = heads.transpose(1, 2).contiguous().view(batch_size, self.max_len, self.d_model)
        
        if self.return_attn:
            return self.dropout(self.fc(multi_head)), attn
        else:
            return self.dropout(self.fc(multi_head))

In [211]:
query = positional_token
key = positional_token
value = positional_token
pad_mask = (ids_list != pad_idx).unsqueeze(-2)

self_attention = MultiHeadAttention(
    n_heads=n_heads, d_model=d_model, max_len=max_len, 
    dropout_p=0., dropout_p_attn=0., 
    return_attn=True
)

multi_attn, attn = self_attention(
    query=query, key=key, value=value, mask=pad_mask
)

In [212]:
multi_attn, multi_attn.shape

(tensor([[[-0.4477, -1.1129, -1.4730, -1.2048],
          [-0.4268, -1.1003, -1.4286, -1.1763],
          [-0.4064, -1.0877, -1.3851, -1.1482],
          [-0.3869, -1.0749, -1.3429, -1.1208],
          [-0.3683, -1.0621, -1.3024, -1.0943]],
 
         [[-0.2813, -0.6753, -0.9107, -0.7402],
          [-0.2601, -0.6627, -0.8658, -0.7114],
          [-0.2392, -0.6501, -0.8214, -0.6828],
          [-0.2189, -0.6375, -0.7779, -0.6548],
          [-0.1993, -0.6247, -0.7358, -0.6274]]], grad_fn=<UnsafeViewBackward>),
 torch.Size([2, 5, 4]))

```python
torch.Size([バッチサイズ, 文章の最大長, 単語のベクトル次元])
``` 

In [213]:
attn, attn.shape

(tensor([[[[0.2603, 0.2533, 0.2465, 0.2399, 0.0000],
           [0.2707, 0.2564, 0.2429, 0.2301, 0.0000],
           [0.2813, 0.2593, 0.2390, 0.2204, 0.0000],
           [0.2920, 0.2620, 0.2351, 0.2109, 0.0000],
           [0.3029, 0.2645, 0.2310, 0.2017, 0.0000]],
 
          [[0.2843, 0.2601, 0.2379, 0.2177, 0.0000],
           [0.3201, 0.2679, 0.2243, 0.1877, 0.0000],
           [0.3570, 0.2734, 0.2093, 0.1603, 0.0000],
           [0.3944, 0.2763, 0.1936, 0.1356, 0.0000],
           [0.4320, 0.2769, 0.1775, 0.1137, 0.0000]]],
 
 
         [[[0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
           [0.2603, 0.2533, 0.2465, 0.2399, 0.0000],
           [0.2707, 0.2564, 0.2429, 0.2301, 0.0000],
           [0.2813, 0.2593, 0.2390, 0.2204, 0.0000],
           [0.2920, 0.2620, 0.2351, 0.2109, 0.0000]],
 
          [[0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
           [0.2843, 0.2601, 0.2379, 0.2177, 0.0000],
           [0.3201, 0.2679, 0.2243, 0.1877, 0.0000],
           [0.3570, 0.2734, 0.2093

```python
torch.Size([バッチサイズ, ヘッド数, 文章の最大長, 文章の最大長,])
```

## Encoder layer

### skip connection & layer norm

attentionから得られたベクトルではなく、positional encodingが済んだベクトルに対して、layer normをかける

In [214]:
positional_token

tensor([[[1., 1., 1., 1.],
         [2., 2., 2., 2.],
         [3., 3., 3., 3.],
         [4., 4., 4., 4.],
         [5., 5., 5., 5.]],

        [[0., 0., 0., 0.],
         [1., 1., 1., 1.],
         [2., 2., 2., 2.],
         [3., 3., 3., 3.],
         [4., 4., 4., 4.]]])

In [215]:
multi_attn

tensor([[[-0.4477, -1.1129, -1.4730, -1.2048],
         [-0.4268, -1.1003, -1.4286, -1.1763],
         [-0.4064, -1.0877, -1.3851, -1.1482],
         [-0.3869, -1.0749, -1.3429, -1.1208],
         [-0.3683, -1.0621, -1.3024, -1.0943]],

        [[-0.2813, -0.6753, -0.9107, -0.7402],
         [-0.2601, -0.6627, -0.8658, -0.7114],
         [-0.2392, -0.6501, -0.8214, -0.6828],
         [-0.2189, -0.6375, -0.7779, -0.6548],
         [-0.1993, -0.6247, -0.7358, -0.6274]]], grad_fn=<UnsafeViewBackward>)

In [216]:
positional_token + multi_attn

tensor([[[ 0.5523, -0.1129, -0.4730, -0.2048],
         [ 1.5732,  0.8997,  0.5714,  0.8237],
         [ 2.5936,  1.9123,  1.6149,  1.8518],
         [ 3.6131,  2.9251,  2.6571,  2.8792],
         [ 4.6317,  3.9379,  3.6976,  3.9057]],

        [[-0.2813, -0.6753, -0.9107, -0.7402],
         [ 0.7399,  0.3373,  0.1342,  0.2886],
         [ 1.7608,  1.3499,  1.1786,  1.3172],
         [ 2.7811,  2.3625,  2.2221,  2.3452],
         [ 3.8007,  3.3753,  3.2642,  3.3726]]], grad_fn=<AddBackward0>)

各単語ベクトルを正規化する

In [217]:
normalize = nn.LayerNorm(normalized_shape=d_model)
normalize(positional_token + multi_attn)

tensor([[[ 1.6220, -0.1413, -1.0958, -0.3849],
         [ 1.6362, -0.1817, -1.0677, -0.3867],
         [ 1.6493, -0.2220, -1.0390, -0.3883],
         [ 1.6611, -0.2614, -1.0102, -0.3896],
         [ 1.6715, -0.2992, -0.9818, -0.3905]],

        [[ 1.6069, -0.1015, -1.1225, -0.3829],
         [ 1.6316, -0.1686, -1.0768, -0.3862],
         [ 1.6542, -0.2382, -1.0271, -0.3888],
         [ 1.6739, -0.3089, -0.9743, -0.3907],
         [ 1.6902, -0.3791, -0.9192, -0.3918]]],
       grad_fn=<NativeLayerNormBackward>)

挙動を確認する

In [218]:
# pytorchのstdだと1/nではなく、1/(n-1)で割っているため、LayerNormと結果が一致しないため、numpyを使用した
mean = positional_token.detach().numpy().mean(axis=2, keepdims=True)
std = positional_token.detach().numpy().std(axis=2, keepdims=True)
eps = 1e-5
normalize.state_dict()['weight'] * (positional_token.detach().numpy() - mean) / (std + eps) + normalize.state_dict()['bias']

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

#### クラス化

In [219]:
class AddNorm(nn.Module):
    def __init__(self, 
                 d_model: int,
                 drop_p: float
                ) -> torch.Tensor:
        super(AddNorm, self).__init__()

        self.dropout = nn.Dropout(drop_p)
        self.normalize = nn.LayerNorm(d_model)  # 各単語ごとに正規化する
        
    def forward(self,
                x: torch.Tensor,
                output: torch.Tensor
               ) -> torch.Tensor:
        # Add & Norm
        x = self.normalize(x + output)
        x = self.dropout(x)
        
        return x

In [220]:
add_norm = AddNorm(d_model=d_model, drop_p=0.5)
add_norm_out = add_norm(positional_token, multi_attn)
add_norm_out

tensor([[[ 0.0000, -0.0000, -0.0000, -0.7699],
         [ 3.2724, -0.0000, -0.0000, -0.7735],
         [ 3.2985, -0.0000, -2.0780, -0.0000],
         [ 0.0000, -0.5227, -2.0204, -0.7791],
         [ 3.3429, -0.0000, -1.9636, -0.0000]],

        [[ 0.0000, -0.0000, -0.0000, -0.0000],
         [ 3.2632, -0.3372, -0.0000, -0.7723],
         [ 3.3083, -0.0000, -0.0000, -0.7776],
         [ 3.3478, -0.6178, -0.0000, -0.7815],
         [ 3.3804, -0.0000, -1.8385, -0.0000]]], grad_fn=<MulBackward0>)

### FeedForward

In [221]:
class FeedForward(nn.Module):
    def __init__(self,
                 in_features: int,
                 hidden_features: int,
                 out_features: int,
                 drop_p: float
                ) -> torch.Tensor:
        super(FeedForward, self).__init__()
        
        self.dropout = nn.Dropout(drop_p)
        self.fc_hidden = nn.Linear(in_features=in_features,
                                   out_features=hidden_features,
                                   bias=False)
        self.fc_out = nn.Linear(in_features=hidden_features,
                                out_features=out_features,
                                bias=False
                               )
        
    def forward(self,
                x: torch.Tensor
               ) -> torch.Tensor:
        x = self.fc_hidden(x)
        x = self.dropout(F.relu(x))
        x = self.fc_out(x)

        return x

In [222]:
ff = FeedForward(in_features=d_model,
                 hidden_features=d_model,
                 out_features=d_model,
                 drop_p=0.5
                )
ff(add_norm_out)

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.9373, -0.8033, -0.2102, -1.2306],
         [ 0.2482, -0.2127, -0.0557, -0.3258],
         [ 0.9271, -0.7946, -0.2079, -1.2172]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.5094, -0.4366, -0.1143, -0.6688],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.9140, -0.7834, -0.2050, -1.2000]]], grad_fn=<UnsafeViewBackward>)

### Encoder layer

#### クラス化

In [223]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 n_heads: int, 
                 d_model: int, 
                 max_len: int,
                 dropout_p: float, 
                 dropout_p_attn: float,
                 dropout_p_ff: float,
                 dropout_p_addnorm: float,
                 return_attn: bool = False
                ):
        super(EncoderLayer, self).__init__()
        
        self.return_attn = return_attn
        
        self.self_attention = MultiHeadAttention(
            n_heads=n_heads, 
            d_model=d_model, 
            max_len=max_len,
            dropout_p=dropout_p, 
            dropout_p_attn=dropout_p_attn,
            return_attn=True
        )
        
        self.ff = FeedForward(
            in_features=d_model,
            hidden_features=d_model,
            out_features=d_model,
            drop_p=dropout_p_ff
        )
        
        self.add_norm_1 = AddNorm(d_model=d_model, drop_p=dropout_p_addnorm)
        self.add_norm_2 = AddNorm(d_model=d_model, drop_p=dropout_p_addnorm)
        
        
    def forward(self, 
                x: torch.Tensor,
                pad_mask: Optional[torch.Tensor] = None
               ) -> torch.Tensor:
        
        query = x.clone()
        key = x.clone()
        value = x.clone()
        output, attn = self.self_attention(
            query, key, value, pad_mask)
        x = self.add_norm_1(x, output)
        x = self.add_norm_2(x, self.ff(x))
        
        if self.return_attn:
            return x, attn
        else:
            return x

In [224]:
positional_token, pad_mask

(tensor([[[1., 1., 1., 1.],
          [2., 2., 2., 2.],
          [3., 3., 3., 3.],
          [4., 4., 4., 4.],
          [5., 5., 5., 5.]],
 
         [[0., 0., 0., 0.],
          [1., 1., 1., 1.],
          [2., 2., 2., 2.],
          [3., 3., 3., 3.],
          [4., 4., 4., 4.]]]),
 tensor([[[ True,  True,  True,  True, False]],
 
         [[ True,  True,  True,  True, False]]]))

In [225]:
encoder_layer = EncoderLayer(
    n_heads=n_heads,
    d_model=d_model,
    max_len=max_len,
    dropout_p=0.,
    dropout_p_attn=0.,
    dropout_p_ff=0.,
    dropout_p_addnorm=0.
)
encoded = encoder_layer(positional_token, pad_mask)
encoded, encoded.shape

(tensor([[[ 1.3781,  0.2613, -1.4065, -0.2329],
          [ 1.4027,  0.2024, -1.3955, -0.2096],
          [ 1.4254,  0.1441, -1.3830, -0.1864],
          [ 1.4459,  0.0871, -1.3693, -0.1636],
          [ 1.4641,  0.0320, -1.3547, -0.1414]],
 
         [[ 1.3517,  0.3201, -1.4159, -0.2560],
          [ 1.3948,  0.2217, -1.3993, -0.2173],
          [ 1.4333,  0.1225, -1.3780, -0.1778],
          [ 1.4665,  0.0245, -1.3526, -0.1384],
          [ 1.4942, -0.0708, -1.3238, -0.0997]]],
        grad_fn=<NativeLayerNormBackward>),
 torch.Size([2, 5, 4]))

```python
torch.Size([バッチサイズ, 文章の最大長, 単語のベクトル次元])
```

## Encoder

EncoderLayerを重ねることで、構成する

### クラス化

In [226]:
class Encoder(nn.Module):
    def __init__(self, 
                 n_layers: int,
                 n_heads: int, 
                 d_model: int, 
                 max_len: int,
                 dropout_p: float = 0.5, 
                 dropout_p_attn: float = 0.5,
                 dropout_p_ff: float = 0.5,
                 dropout_p_addnorm: float = 0.5,
                 return_attns: bool = False
                ):
        super(Encoder, self).__init__()
        self.return_attns = return_attns
        
        self.layer_stack = nn.ModuleList([
            EncoderLayer(n_heads=n_heads,
                         d_model=d_model,
                         max_len=max_len,
                         dropout_p=dropout_p,
                         dropout_p_attn=dropout_p_attn,
                         dropout_p_ff=dropout_p_ff,
                         dropout_p_addnorm=dropout_p_addnorm,
                         return_attn=True)
            for _ in range(n_layers)
        ])
        
        
    def forward(self, x, pad_mask):
        enc_self_attn_list = []
        
        for enc_layer in self.layer_stack:
            x, enc_self_attn = enc_layer(x, pad_mask)
            enc_self_attn_list += [enc_self_attn] if self.return_attns else []

        if self.return_attns:
            return x, enc_self_attn_list
        return x

In [227]:
encoder = Encoder(n_layers=n_layers, n_heads=n_heads, d_model=d_model, max_len=max_len)
encoder_output = encoder(positional_token, pad_mask)
encoder_output

tensor([[[-2.9388,  0.0000,  2.0914, -0.0000],
         [-0.0000,  2.9104,  0.7969, -1.9583],
         [ 0.0000, -2.9936,  0.0000, -0.1047],
         [-3.3864,  1.2969,  0.0000,  0.0000],
         [-0.0000,  0.0000,  2.7166, -0.0000]],

        [[-0.0000,  1.4056,  1.4056,  0.6066],
         [ 0.0000,  0.0000, -0.0000, -0.0000],
         [-0.5617, -1.5538, -1.2909,  0.0000],
         [-3.0486,  0.0000,  2.5443,  0.0000],
         [ 0.0000,  2.8313, -0.0000, -0.0000]]], grad_fn=<MulBackward0>)

## Masked Multi-Head Attention

### target mask

self attentionで、未来の情報を参照しないようにする  
e.g.)「<span style="color: red; ">私</span>」はそのさきの情報は、参照してはいけないため、「は」「AI」「です」「。」は、Falseになっている

![](./image/target_mask.png)

In [228]:
target_mask = (torch.triu(torch.ones(max_len, max_len), diagonal=1) == 0)
target_mask, target_mask.shape

(tensor([[ True, False, False, False, False],
         [ True,  True, False, False, False],
         [ True,  True,  True, False, False],
         [ True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True]]),
 torch.Size([5, 5]))

In [229]:
attn.shape

torch.Size([2, 2, 5, 5])

attentionのshapeに合わせて、target_maskを変形する

In [230]:
target_mask = target_mask.unsqueeze(0).unsqueeze(1)
target_mask, target_mask.shape

(tensor([[[[ True, False, False, False, False],
           [ True,  True, False, False, False],
           [ True,  True,  True, False, False],
           [ True,  True,  True,  True, False],
           [ True,  True,  True,  True,  True]]]]),
 torch.Size([1, 1, 5, 5]))

In [231]:
attn.masked_fill(target_mask == False, -1e9)

tensor([[[[ 2.6026e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [ 2.7068e-01,  2.5640e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [ 2.8127e-01,  2.5930e-01,  2.3905e-01, -1.0000e+09, -1.0000e+09],
          [ 2.9201e-01,  2.6200e-01,  2.3508e-01,  2.1092e-01, -1.0000e+09],
          [ 3.0287e-01,  2.6448e-01,  2.3096e-01,  2.0168e-01,  0.0000e+00]],

         [[ 2.8428e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [ 3.2011e-01,  2.6793e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [ 3.5699e-01,  2.7337e-01,  2.0934e-01, -1.0000e+09, -1.0000e+09],
          [ 3.9444e-01,  2.7634e-01,  1.9359e-01,  1.3563e-01, -1.0000e+09],
          [ 4.3196e-01,  2.7686e-01,  1.7745e-01,  1.1374e-01,  0.0000e+00]]],


        [[[ 2.5000e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [ 2.6026e-01,  2.5330e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [ 2.7068e-01,  2.5640e-01,  2.4287e-01, -1.0000e+09, -1.0000

MultiHeadAttentionを使用して、Masked Multi-Head Attentionを実装する

In [232]:
pad_mask = (ids_list != pad_idx).unsqueeze(-2)
target_mask = (torch.triu(torch.ones(max_len, max_len), diagonal=1) == 0)

In [233]:
pad_mask

tensor([[[ True,  True,  True,  True, False]],

        [[ True,  True,  True,  True, False]]])

In [234]:
target_mask

tensor([[ True, False, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True,  True, False, False],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True]])

In [235]:
pad_mask.shape, target_mask.shape

(torch.Size([2, 1, 5]), torch.Size([5, 5]))

In [236]:
mask = pad_mask & target_mask
mask, mask.shape

(tensor([[[ True, False, False, False, False],
          [ True,  True, False, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True, False],
          [ True,  True,  True,  True, False]],
 
         [[ True, False, False, False, False],
          [ True,  True, False, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True, False],
          [ True,  True,  True,  True, False]]]),
 torch.Size([2, 5, 5]))

In [237]:
query = positional_token
key = positional_token
value = positional_token

mask = pad_mask & target_mask

self_attention = MultiHeadAttention(
    n_heads=n_heads, d_model=d_model, max_len=max_len, 
    dropout_p=0.0, dropout_p_attn=0.0, return_attn=True
)

masked_multi_attn, masked_attn = self_attention(
    query=query, key=key, value=value, mask=mask
)

In [238]:
masked_multi_attn

tensor([[[ 0.1782, -0.6484,  0.1852,  0.3157],
         [ 0.1853, -0.9692,  0.2789,  0.4743],
         [ 0.0536, -1.2918,  0.3767,  0.6379],
         [-0.1996, -1.6405,  0.4852,  0.8178],
         [-0.2814, -1.6624,  0.4937,  0.8311]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0474, -0.3222,  0.0931,  0.1581],
         [-0.0351, -0.6412,  0.1885,  0.3184],
         [-0.2668, -0.9738,  0.2918,  0.4898],
         [-0.3778, -0.9921,  0.3000,  0.5021]]], grad_fn=<UnsafeViewBackward>)

In [239]:
masked_multi_attn.shape

torch.Size([2, 5, 4])

## Decoder layer

### クラス化

In [240]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 n_heads: int, 
                 d_model: int, 
                 max_len: int,
                 dropout_p: float, 
                 dropout_p_attn: float,
                 dropout_p_ff: float,
                 dropout_p_addnorm: float,
                 return_attn: bool = False
                ):
        super(DecoderLayer, self).__init__()
        
        self.return_attn = return_attn
        
        self.self_attention = MultiHeadAttention(
            n_heads=n_heads, 
            d_model=d_model, 
            max_len=max_len,
            dropout_p=dropout_p, 
            dropout_p_attn=dropout_p_attn,
            return_attn=True
        )
        
        self.source_target_attention = MultiHeadAttention(
            n_heads=n_heads, 
            d_model=d_model, 
            max_len=max_len,
            dropout_p=dropout_p, 
            dropout_p_attn=dropout_p_attn,
            return_attn=True
        )
        
        self.ff = FeedForward(
            in_features=d_model,
            hidden_features=d_model,
            out_features=d_model,
            drop_p=dropout_p_ff
        )
        
        self.add_norm_1 = AddNorm(d_model=d_model, drop_p=dropout_p_addnorm)
        self.add_norm_2 = AddNorm(d_model=d_model, drop_p=dropout_p_addnorm)
        self.add_norm_3 = AddNorm(d_model=d_model, drop_p=dropout_p_addnorm)
        
    def forward(self,
                x: torch.Tensor,  # torch.Size([batch_size, seq_length, word_dim])
                encoder_out: torch.Tensor,  # torch.Size([batch_size, seq_length, word_dim]) 
                pad_mask: Optional[torch.Tensor] = None,
                target_mask: Optional[torch.Tensor] = None
               ) -> torch.Tensor:
        
        query = x.clone()
        key = x.clone()
        value = x.clone()
        
        self_attn_output, self_attn = self.self_attention(
            query, key, value, pad_mask & target_mask)
        
        source_target_attn_output, source_target_attn = self.source_target_attention(
            self_attn_output, encoder_out, encoder_out, pad_mask)
        
        x = self.add_norm_1(x, self_attn_output)
        x = self.add_norm_2(x, source_target_attn_output)
        x = self.add_norm_3(x, self.ff(x))
        
        if self.return_attn:
            return x, self_attn, source_target_attn
        else:
            return x

In [241]:
decoder_layer = DecoderLayer(n_heads=n_heads,
                             d_model=d_model,
                             max_len=max_len,
                             dropout_p=0.,
                             dropout_p_attn=0.,
                             dropout_p_ff=0.,
                             dropout_p_addnorm=0.)

decoder_layer(positional_token, encoded, pad_mask, target_mask)

tensor([[[-0.2317,  1.0254, -1.5326,  0.7389],
         [-0.2662,  1.0127, -1.5189,  0.7724],
         [-0.3055,  0.9972, -1.5019,  0.8103],
         [-0.3129,  0.9942, -1.4986,  0.8173],
         [-0.3031,  0.9982, -1.5030,  0.8079]],

        [[-0.9660, -0.5759, -0.1085,  1.6503],
         [-0.2877,  1.0044, -1.5098,  0.7931],
         [-0.3532,  0.9771, -1.4794,  0.8555],
         [-0.3691,  0.9700, -1.4714,  0.8705],
         [-0.3563,  0.9757, -1.4778,  0.8585]]],
       grad_fn=<NativeLayerNormBackward>)

## Decoder

### クラス化

In [76]:
class Decoder(nn.Module):
    def __init__(self, 
                 n_layers: int,
                 n_heads: int, 
                 d_model: int, 
                 max_len: int,
                 dropout_p: float = 0.5, 
                 dropout_p_attn: float = 0.5,
                 dropout_p_ff: float = 0.5,
                 dropout_p_addnorm: float = 0.5,
                 return_attns: bool = False
                ):
        super(Decoder, self).__init__()
        self.return_attns = return_attns
        
        self.layer_stack = nn.ModuleList([
            DecoderLayer(n_heads=n_heads,
                         d_model=d_model,
                         max_len=max_len,
                         dropout_p=dropout_p,
                         dropout_p_attn=dropout_p_attn,
                         dropout_p_ff=dropout_p_ff,
                         dropout_p_addnorm=dropout_p_addnorm,
                         return_attn=True)
            for _ in range(n_layers)
        ])
        
        
    def forward(self,
                x: torch.Tensor,  # torch.Size([batch_size, seq_length, word_dim])
                encoder_out: torch.Tensor,  # torch.Size([batch_size, seq_length, word_dim]) 
                pad_mask: Optional[torch.Tensor] = None,
                target_mask: Optional[torch.Tensor] = None
               ) -> torch.Tensor:
        dec_self_attn_list = []
        dec_source_target_attn_list = []
        
        for dec_layer in self.layer_stack:
            x, dec_self_attn, dec_source_target_attn = dec_layer(
                x, encoder_out, pad_mask, target_mask)
            dec_self_attn_list += [dec_self_attn] if self.return_attns else []
            dec_source_target_attn_list += \
                [dec_source_target_attn] if self.return_attns else []
        
        if self.return_attns:
            return x, dec_self_attn_list, dec_source_target_attn_list
        
        else:
            return x

In [77]:
decoder = Decoder(n_layers=n_layers, n_heads=n_heads, d_model=d_model, max_len=max_len)
decoder_output = decoder(positional_token, encoder_output, pad_mask, target_mask)
decoder_output

tensor([[[-0.0000e+00, -1.3789e+00,  4.4855e+00,  1.8847e+00, -9.6453e-02,
          -7.1868e-01, -1.6286e+00, -0.0000e+00],
         [ 1.4189e+00, -0.0000e+00,  5.0310e-01,  6.7367e-01,  9.3109e-02,
           0.0000e+00,  6.5735e-01,  7.0875e-01],
         [-3.5015e+00, -4.0627e-01, -0.0000e+00, -0.0000e+00,  0.0000e+00,
           1.5715e-01,  4.2882e+00, -0.0000e+00],
         [ 0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -1.4942e+00,
           1.3587e-01, -0.0000e+00, -0.0000e+00],
         [ 5.7781e-01, -0.0000e+00,  3.2293e-01,  4.0367e-01,  1.1979e-01,
           1.6545e+00,  9.5027e-01,  0.0000e+00],
         [-1.3485e+00, -0.0000e+00, -6.5392e-01,  0.0000e+00, -0.0000e+00,
          -0.0000e+00,  0.0000e+00, -0.0000e+00],
         [ 0.0000e+00,  3.2198e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00,
          -0.0000e+00, -0.0000e+00,  0.0000e+00],
         [-6.6185e-01,  6.4638e-01,  0.0000e+00, -2.8641e+00,  6.9052e-01,
          -0.0000e+00,  4.2400e+00, -1.0438e+00],


## Transformer

In [361]:
class Transformer(nn.Module):
    def __init__(self,
                 n_layers: int,
                 n_heads: int, 
                 d_model: int,
                 source_vocab_size: int,
                 target_vocab_size: int,
                 source_pad_idx: int,
                 target_pad_idx: int,
                 max_len: int,
                 dropout_p: float = 0.1, 
                 dropout_p_attn: float = 0.1,
                 dropout_p_ff: float = 0.1,
                 dropout_p_addnorm: float = 0.1
                ):
        super(Transformer, self).__init__()
        
        self.source_pad_idx = source_pad_idx
        self.target_pad_idx = target_pad_idx
        self.max_len = max_len
        
        self.source_token_embeding = nn.Embedding(
            num_embeddings=source_vocab_size,
            embedding_dim=d_model
        )
        
        self.source_positional_encoder = PositionalEncoding(
            d_model=d_model, dropout_p=dropout_p, max_len=max_len
        )
        
        self.target_token_embeding = nn.Embedding(
            num_embeddings=target_vocab_size,
            embedding_dim=d_model
        )
        
        self.target_positional_encoder = PositionalEncoding(
            d_model=d_model, dropout_p=dropout_p, max_len=max_len
        )
        
        self.encoder = Encoder(
            n_layers=n_layers, n_heads=n_heads, 
            d_model=d_model, max_len=max_len
        )
        
        self.decoder = Decoder(
            n_layers=n_layers, n_heads=n_heads, 
            d_model=d_model, max_len=max_len
        )
        
        self.fc = nn.Linear(
            in_features=d_model, out_features=target_vocab_size, bias=False)
    
    def get_pad_mask(self, x, pad_idx):
        return (x != pad_idx).unsqueeze(-2)
    
    def get_target_mask(self, target):
        return (torch.triu(torch.ones(self.max_len, self.max_len), 
                           diagonal=1) == 0).unsqueeze(0)
    
    def forward(self, source, target):
        source_pad_mask = self.get_pad_mask(source, self.source_pad_idx)
        source = self.source_token_embeding(source)
        source = self.source_positional_encoder(source)
        encoder_output = self.encoder(source, source_pad_mask)
        
        target_pad_mask = self.get_pad_mask(target, self.target_pad_idx)
        target_mask = self.get_target_mask(target)
        print(target_pad_mask.shape, target_mask.shape)
        target = self.target_token_embeding(target)
        target = self.target_positional_encoder(target)
        print(target.shape)
        decoder_out = self.decoder(target, encoder_output, 
                                   target_pad_mask, target_mask)
        
        return self.fc(decoder_out)
    
    def encode(self, source):
        source_pad_mask = self.get_pad_mask(source, self.source_pad_idx)
        source = self.source_token_embeding(source)
        source = self.source_positional_encoder(source)
        encoder_output = self.encoder(source, source_pad_mask)
        return encoder_output
    
    def decode(self, encoder_output, target):
        target_pad_mask = self.get_pad_mask(target, self.target_pad_idx)
        target_mask = self.get_target_mask(target)
        target = self.target_token_embeding(target)
        target = self.target_positional_encoder(target)
        decoder_out = self.decoder(target, encoder_output, 
                                   target_pad_mask, target_mask)
        return F.softmax(decoder_out, dim=-1)

In [250]:
max_len = 10
d_model = 12
batch_size = 3
n_heads = 2
n_layers = 5

pad = '<pad>'

source_list = [['私', 'は', 'AI', 'です', '。'],
               ['私', 'が', 'AI', 'だ', '。'],
               ['私', 'の', 'AI', 'は', '最強', 'です', '。']
              ]

target_list = [['<start>', 'I', 'am', 'AI', '.', '<end>'],
               ['<start>', 'I', 'am', 'AI', '.', '<end>'],
               ['<start>', 'My', 'AI', 'is', 'the', 'strongest', '.', '<end>']
              ]

target_list = [['<start>', '<end>'],
               ['<start>', '<end>'],
               ['<start>', '<end>']
              ]

target_in_list = [words[1:] for words in target_list]
target_out_list = [words[:-1] for words in target_list]

source_list = [
    [words[i] if i < len(words) else pad for i in range(max_len)]
    for words in source_list
]

target_in_list = [
    [words[i] if i < len(words) else pad for i in range(max_len)]
    for words in target_in_list
]

target_out_list = [
    [words[i] if i < len(words) else pad for i in range(max_len)]
    for words in target_out_list
]

source_vocab_size = len(list(set(sum(source_list, []))))
target_vocab_size = len(list(set(sum(target_in_list+target_out_list, []))))
word_list = source_list + target_list

source_le = preprocessing.LabelEncoder()
source_le.fit(sum(source_list, []))

target_le = preprocessing.LabelEncoder()
target_le.fit(sum(target_in_list+target_out_list, []))

source_ids_list = torch.tensor([source_le.transform(w) for w in source_list])
target_in_ids_list = torch.tensor([target_le.transform(w) for w in target_in_list])
target_out_ids_list = torch.tensor([target_le.transform(w) for w in target_out_list])

transformer = Transformer(n_layers=n_layers,
                          n_heads=n_heads,
                          d_model=d_model,
                          source_vocab_size=source_vocab_size,
                          target_vocab_size=target_vocab_size,
                          source_pad_idx=source_le.transform([pad])[0],
                          target_pad_idx=target_le.transform([pad])[0],
                          max_len=max_len
                         )

In [251]:
target_le.classes_, target_vocab_size

(array(['<end>', '<pad>', '<start>'], dtype='<U7'), 3)

In [252]:
source_ids_list, target_in_ids_list, target_out_ids_list

(tensor([[9, 7, 1, 5, 2, 0, 0, 0, 0, 0],
         [9, 3, 1, 4, 2, 0, 0, 0, 0, 0],
         [9, 6, 1, 7, 8, 5, 2, 0, 0, 0]]),
 tensor([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 tensor([[2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [2, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))

In [253]:
output = transformer(source_ids_list, target_in_ids_list)
output, output.shape

(tensor([[[ 1.2595,  0.7338,  0.3748],
          [ 0.6248,  0.2551, -0.8388],
          [-0.5289, -1.3264, -1.2340],
          [ 0.2748,  1.0528, -0.0491],
          [ 0.5552,  0.2505,  0.7270],
          [ 0.0578,  0.9510,  0.8986],
          [ 0.1808, -0.2132,  0.7288],
          [-0.6409,  0.1907, -0.7351],
          [ 1.1712,  0.5595,  0.4800],
          [ 0.2909, -0.1832,  1.5794]],
 
         [[-0.0765, -0.4302, -0.5676],
          [-0.2655, -1.0443,  0.9522],
          [ 0.1734, -1.3008,  1.1382],
          [ 0.3084,  0.0808,  0.8367],
          [ 1.2538, -0.5137, -0.7076],
          [-0.8560, -0.6670, -1.3404],
          [ 0.0856, -1.7685,  2.0288],
          [-0.0675, -1.0991, -0.5858],
          [-0.9254, -0.3795,  1.3626],
          [ 0.3020, -0.4901,  1.7900]],
 
         [[-1.0691, -0.9471, -0.1764],
          [ 0.4395, -0.9465, -1.5304],
          [ 0.5746,  1.1230, -1.4655],
          [-0.7733, -0.7691,  0.6643],
          [ 1.0392,  0.4303, -0.5764],
          [ 0.2144,

torch.log(best_k2_probs).view(beam_size, -1)```python
torch.Size([バッチサイズ, 文章の最大長, target側のvocab数])
```

## loss function

### calculate_loss

In [254]:
target_out_ids_list, target_out_ids_list.shape

(tensor([[2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [2, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 torch.Size([3, 10]))

[cross_entropyの仕様](https://pytorch.org/docs/stable/nn.functional.html#cross-entropy)上、文章の最大長次元とtarget側のvocab数次元を入れ替える必要がある

In [255]:
F.cross_entropy(output.transpose(1, 2), target_out_ids_list)

tensor(1.5081, grad_fn=<NllLoss2DBackward>)

In [256]:
def calculate_loss(pred: torch.Tensor,  # torch.Size([batch, max_len, target_vocab_len]) 
                   t: torch.Tensor  # torch.Size([batch, max_len])
                  ) -> torch.Tensor:
    return F.cross_entropy(pred.transpose(1, 2), t)

### calculte_performance

In [257]:
pad_idx = target_le.transform([pad])[0]
pad_idx

1

In [258]:
output.max(dim=-1)

torch.return_types.max(
values=tensor([[ 1.2595,  0.6248, -0.5289,  1.0528,  0.7270,  0.9510,  0.7288,  0.1907,
          1.1712,  1.5794],
        [-0.0765,  0.9522,  1.1382,  0.8367,  1.2538, -0.6670,  2.0288, -0.0675,
          1.3626,  1.7900],
        [-0.1764,  0.4395,  1.1230,  0.6643,  1.0392,  0.2144, -0.1830, -0.8288,
         -0.4467,  1.6629]], grad_fn=<MaxBackward0>),
indices=tensor([[0, 0, 0, 1, 2, 1, 2, 1, 0, 2],
        [0, 2, 2, 2, 0, 1, 2, 0, 2, 2],
        [2, 0, 1, 2, 0, 0, 0, 0, 2, 1]]))

In [259]:
prediction = torch.max(output, dim=-1)[1]
prediction

tensor([[0, 0, 0, 1, 2, 1, 2, 1, 0, 2],
        [0, 2, 2, 2, 0, 1, 2, 0, 2, 2],
        [2, 0, 1, 2, 0, 0, 0, 0, 2, 1]])

In [260]:
prediction, target_out_ids_list

(tensor([[0, 0, 0, 1, 2, 1, 2, 1, 0, 2],
         [0, 2, 2, 2, 0, 1, 2, 0, 2, 2],
         [2, 0, 1, 2, 0, 0, 0, 0, 2, 1]]),
 tensor([[2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [2, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))

In [261]:
n_corrects = prediction.eq(
    target_out_ids_list
).masked_select(
    target_out_ids_list != pad_idx
).sum().item()
n_corrects

1

In [262]:
n_words = len(target_out_ids_list.masked_select(target_out_ids_list != pad_idx))
n_words

3

In [263]:
def calculate_performance(pred: torch.Tensor, 
                          t: torch.Tensor, 
                          target_pad_idx: int
                         ) -> Tuple[torch.Tensor, int, int]:
    loss = calculate_loss(pred, t)
    
    prediction = torch.max(pred, dim=-1)[1]
    n_corrects = prediction.eq(t).masked_select(t != target_pad_idx).sum().item()
    n_words = len(t.masked_select(t != target_pad_idx))
    
    return loss, n_corrects, n_words

In [264]:
calculate_performance(output, target_out_ids_list, pad_idx)

(tensor(1.5081, grad_fn=<NllLoss2DBackward>), 1, 3)

# DataLoader

In [265]:
class ShortenDataset(Dataset):
    def __init__(self, 
                 originals: list, 
                 shorts: list,
                 max_len: int
                ):
        self.pad_id = 0
        self.macron_id = 27
        self.start_id = 28
        self.end_id = 29
        
        transform = kakasi()
        self.max_len = max_len
             
        self.originals = [
            list(map(self.transform, 
                     list(''.join([item['kunrei'].replace('#', '-') 
                                   for item in transform.convert(text.replace('ー', '#'))]))
                    )) for text in originals
        ]
        
        self.shorts = [
            list(map(self.transform, 
                     list(''.join([item['kunrei'].replace('#', '-') 
                                   for item in transform.convert(text.replace('ー', '#'))]))
                    )) for text in shorts
        ]
        
        self.originals = list(map(self.padding, self.originals))
        self.shorts = list(map(self.shift_right, self.shorts))
        
        
    def __len__(self):
        return len(self.originals)
        
    def __getitem__(self, 
                    index: int
                   ) -> Tuple[torch.Tensor]:
        return torch.tensor(self.originals[index]), torch.tensor(self.shorts[index])
    
    def transform(self, alphabet: List[str]) -> List[str]:
        if alphabet.isalpha():
            return ord(alphabet) - ord('a') + 1
        elif alphabet == '-':
            return self.macron_id
        elif alphabet == "'":
            return ord('n') - ord('a') + 1
        
    def padding(self, alphabet_list: List[str]) -> List[str]:
        return alphabet_list + [self.pad_id for _ in range(self.max_len-len(alphabet_list))]
    
    def shift_right(self, alphabet_list: List[str]) -> List[str]:
        return [self.start_id] + alphabet_list + [self.end_id] + [self.pad_id for _ in range(self.max_len-len(alphabet_list)-2 + 1)]

In [266]:
df = pd.read_csv('data/train_manual.csv', encoding='SHIFT-JIS')
df.head()

,short,original
0,アイコ,アイスコーヒー
1,アイシン,アイシン精機
2,赤プリ,グランドプリンスホテル赤坂
3,アキバ,秋葉原
4,アクエリ,アクエリアス


In [267]:
dataset = ShortenDataset(df['original'].tolist(), df['short'].tolist(), 50)
len(dataset)

397

In [268]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [269]:
for x, t in dataloader:
    print(x.shape, t.shape)
    print(x)
    print(t[:,:-1])
    print(t[:,1:])
    break

torch.Size([4, 50]) torch.Size([4, 51])
tensor([[16, 15, 11,  5, 20, 20, 15, 19, 21, 20,  5, 27, 19, 25, 15, 14,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 5,  1, 27, 13,  1, 14,  5, 26,  9, 13,  5, 14, 20, 15,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [14,  1, 14,  2,  1, 27, 16, 21, 18,  5, 27, 19, 21,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 2, 21, 18,  1, 11, 11, 21, 18,  1,  7, 21, 27, 14,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
tensor([[28, 16, 15, 11,  5, 19, 21, 20,  5, 29

In [270]:
# 動作確認
for x, t in dataloader:
    pass

# optimizer

In [271]:
class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''

    def __init__(self, optimizer, lr_mul, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.lr_mul = lr_mul
        self.d_model = d_model
        self.n_warmup_steps = n_warmup_steps
        self.n_steps = 0


    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()


    def zero_grad(self):
        "Zero out the gradients with the inner optimizer"
        self._optimizer.zero_grad()


    def _get_lr_scale(self):
        d_model = self.d_model
        n_steps, n_warmup_steps = self.n_steps, self.n_warmup_steps
        return (d_model ** -0.5) * min(n_steps ** (-0.5), 
                                       n_steps * n_warmup_steps ** (-1.5))


    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_steps += 1
        lr = self.lr_mul * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

# 学習

In [1]:
import numpy as np
import pandas as pd
import os
import math
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from pykakasi import kakasi

from typing import Optional, Tuple, List
from tqdm.notebook import tqdm
import argparse

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout_p: float, max_len: int):
        super(PositionalEncoding, self).__init__()
        
        self.dropout = nn.Dropout(p=dropout_p)
        # Not a parameter
        self.register_buffer('pe', self._pe_table(d_model, max_len))

    def _pe_table(self, d_model: int, max_len: int) -> torch.Tensor:
        pos = torch.arange(0, max_len).unsqueeze(1)
        pe_denominator = torch.exp(
            torch.arange(0, d_model, 2) * math.log(10000) / d_model
        )

        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(pos / pe_denominator)  # 偶数項にはsin
        pe[:, 1::2] = torch.cos(pos / pe_denominator)  # 奇数項にはcos

        pe = pe.unsqueeze(0)  # バッチに対応するため
        return pe

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.dropout(x + self.pe[:, :x.size(1)].clone().detach())

In [3]:
class ScaledDotProductAttention(nn.Module):
    
    def __init__(self, scale: int, drop_p: float):
        super(ScaledDotProductAttention, self).__init__()
        
        self.scale = math.sqrt(scale)
        self.dropout = nn.Dropout(drop_p)
        
    def forward(self, 
                Q: torch.Tensor,  # torch.Size([batch_size, n_heads, seq_length, word_dim]) 
                K: torch.Tensor,  # torch.Size([batch_size, n_heads, seq_length, word_dim])
                V: torch.Tensor,  # torch.Size([batch_size, n_heads, seq_length, word_dim])
                mask:Optional[torch.Tensor] = None  # torch.Size([[batch_size, n_heads, seq_length, word_dim]]) decoderで使用する
               ) -> torch.Tensor:  
        attn = torch.matmul(Q / self.scale, K.transpose(2,3))
        
        if mask is not None:
            attn = attn.masked_fill(mask == False, -1e9)
            
        attn = self.dropout(F.softmax(attn, dim=-1))
        output = torch.matmul(attn, V)
        
        return output, attn

In [4]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, 
                 n_heads: int, 
                 d_model: int, 
                 max_len: int,
                 dropout_p: float, 
                 dropout_p_attn: float,
                 return_attn: bool = False):
        super(MultiHeadAttention, self).__init__()
        
        self.n_heads = n_heads
        self.d_model = d_model
        self.max_len = max_len
        self.return_attn = return_attn
        
        self.W_q = nn.Linear(in_features=d_model,
                             out_features=d_model,
                             bias=False)
        self.W_k = nn.Linear(in_features=d_model,
                             out_features=d_model,
                             bias=False)
        self.W_v = nn.Linear(in_features=d_model,
                             out_features=d_model,
                             bias=False)
        self.fc = nn.Linear(in_features=d_model,
                            out_features=d_model,
                            bias=False)
        
        d_k = d_model // n_heads
        self.attention = ScaledDotProductAttention(
            scale=d_k, drop_p=dropout_p_attn
        )
        
        self.dropout = nn.Dropout(dropout_p)
        
    def forward(self, 
                query: torch.Tensor,
                key: torch.Tensor,
                value: torch.Tensor,
                mask: Optional[torch.Tensor] = None
               ) -> torch.Tensor:
        batch_size = query.size(0)
        
        Q_single = self.W_q(query)
        K_single = self.W_k(key)
        V_single = self.W_v(value)
        
        Q_multi = Q_single.contiguous().view(
            batch_size, -1, self.n_heads, self.d_model//self.n_heads
        ).transpose(1,2)
        K_multi = K_single.contiguous().view(
            batch_size, -1, self.n_heads, self.d_model//self.n_heads
        ).transpose(1,2)
        V_multi = V_single.contiguous().view(
            batch_size, -1, self.n_heads, self.d_model//self.n_heads
        ).transpose(1,2)
        
        if mask is not None:
            mask = mask.unsqueeze(1)  # heads次元の追加
        
        # Scaled Dot-Product Attention
        heads, attn = self.attention(Q_multi, K_multi, V_multi, mask)
        
        # Concat
        multi_head = heads.transpose(1, 2).contiguous().view(batch_size, self.max_len, self.d_model)
        
        if self.return_attn:
            return self.dropout(self.fc(multi_head)), attn
        else:
            return self.dropout(self.fc(multi_head))

In [5]:
class AddNorm(nn.Module):
    def __init__(self, 
                 d_model: int,
                 drop_p: float
                ) -> torch.Tensor:
        super(AddNorm, self).__init__()

        self.dropout = nn.Dropout(drop_p)
        self.normalize = nn.LayerNorm(d_model)  # 各単語ごとに正規化する
        
    def forward(self,
                x: torch.Tensor,
                output: torch.Tensor
               ) -> torch.Tensor:
        # Add & Norm
        x = self.normalize(x + output)
        x = self.dropout(x)
        
        return x

In [6]:
class FeedForward(nn.Module):
    def __init__(self,
                 in_features: int,
                 hidden_features: int,
                 out_features: int,
                 drop_p: float
                ) -> torch.Tensor:
        super(FeedForward, self).__init__()
        
        self.dropout = nn.Dropout(drop_p)
        self.fc_hidden = nn.Linear(in_features=in_features,
                                   out_features=hidden_features,
                                   bias=False)
        self.fc_out = nn.Linear(in_features=hidden_features,
                                out_features=out_features,
                                bias=False
                               )
        
    def forward(self,
                x: torch.Tensor
               ) -> torch.Tensor:
        x = self.fc_hidden(x)
        x = self.dropout(F.relu(x))
        x = self.fc_out(x)

        return x

In [7]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 n_heads: int, 
                 d_model: int, 
                 max_len: int,
                 dropout_p: float, 
                 dropout_p_attn: float,
                 dropout_p_ff: float,
                 dropout_p_addnorm: float,
                 return_attn: bool = False
                ):
        super(EncoderLayer, self).__init__()
        
        self.return_attn = return_attn
        
        self.self_attention = MultiHeadAttention(
            n_heads=n_heads, 
            d_model=d_model, 
            max_len=max_len,
            dropout_p=dropout_p, 
            dropout_p_attn=dropout_p_attn,
            return_attn=True
        )
        
        self.ff = FeedForward(
            in_features=d_model,
            hidden_features=d_model,
            out_features=d_model,
            drop_p=dropout_p_ff
        )
        
        self.add_norm_1 = AddNorm(d_model=d_model, drop_p=dropout_p_addnorm)
        self.add_norm_2 = AddNorm(d_model=d_model, drop_p=dropout_p_addnorm)
        
        
    def forward(self, 
                x: torch.Tensor,
                pad_mask: Optional[torch.Tensor] = None
               ) -> torch.Tensor:
        
        query = x.clone()
        key = x.clone()
        value = x.clone()
        output, attn = self.self_attention(
            query, key, value, pad_mask)
        x = self.add_norm_1(x, output)
        x = self.add_norm_2(x, self.ff(x))
        
        if self.return_attn:
            return x, attn
        else:
            return x

In [8]:
class Encoder(nn.Module):
    def __init__(self, 
                 n_layers: int,
                 n_heads: int, 
                 d_model: int, 
                 max_len: int,
                 dropout_p: float = 0.5, 
                 dropout_p_attn: float = 0.5,
                 dropout_p_ff: float = 0.5,
                 dropout_p_addnorm: float = 0.5,
                 return_attns: bool = False
                ):
        super(Encoder, self).__init__()
        self.return_attns = return_attns
        
        self.layer_stack = nn.ModuleList([
            EncoderLayer(n_heads=n_heads,
                         d_model=d_model,
                         max_len=max_len,
                         dropout_p=dropout_p,
                         dropout_p_attn=dropout_p_attn,
                         dropout_p_ff=dropout_p_ff,
                         dropout_p_addnorm=dropout_p_addnorm,
                         return_attn=True)
            for _ in range(n_layers)
        ])
        
        
    def forward(self, x, pad_mask):
        enc_self_attn_list = []
        
        for enc_layer in self.layer_stack:
            x, enc_self_attn = enc_layer(x, pad_mask)
            enc_self_attn_list += [enc_self_attn] if self.return_attns else []

        if self.return_attns:
            return x, enc_self_attn_list
        return x

In [9]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 n_heads: int, 
                 d_model: int, 
                 max_len: int,
                 dropout_p: float, 
                 dropout_p_attn: float,
                 dropout_p_ff: float,
                 dropout_p_addnorm: float,
                 return_attn: bool = False
                ):
        super(DecoderLayer, self).__init__()
        
        self.return_attn = return_attn
        
        self.self_attention = MultiHeadAttention(
            n_heads=n_heads, 
            d_model=d_model, 
            max_len=max_len,
            dropout_p=dropout_p, 
            dropout_p_attn=dropout_p_attn,
            return_attn=True
        )
        
        self.source_target_attention = MultiHeadAttention(
            n_heads=n_heads, 
            d_model=d_model, 
            max_len=max_len,
            dropout_p=dropout_p, 
            dropout_p_attn=dropout_p_attn,
            return_attn=True
        )
        
        self.ff = FeedForward(
            in_features=d_model,
            hidden_features=d_model,
            out_features=d_model,
            drop_p=dropout_p_ff
        )
        
        self.add_norm_1 = AddNorm(d_model=d_model, drop_p=dropout_p_addnorm)
        self.add_norm_2 = AddNorm(d_model=d_model, drop_p=dropout_p_addnorm)
        self.add_norm_3 = AddNorm(d_model=d_model, drop_p=dropout_p_addnorm)
        
    def forward(self,
                x: torch.Tensor,  # torch.Size([batch_size, seq_length, word_dim])
                encoder_out: torch.Tensor,  # torch.Size([batch_size, seq_length, word_dim]) 
                pad_mask: Optional[torch.Tensor] = None,
                target_mask: Optional[torch.Tensor] = None
               ) -> torch.Tensor:
        
        query = x.clone()
        key = x.clone()
        value = x.clone()
        
        self_attn_output, self_attn = self.self_attention(
            query, key, value, pad_mask & target_mask)
        
        source_target_attn_output, source_target_attn = self.source_target_attention(
            self_attn_output, encoder_out, encoder_out, pad_mask)
        
        x = self.add_norm_1(x, self_attn_output)
        x = self.add_norm_2(x, source_target_attn_output)
        x = self.add_norm_3(x, self.ff(x))
        
        if self.return_attn:
            return x, self_attn, source_target_attn
        else:
            return x

In [10]:
class Decoder(nn.Module):
    def __init__(self, 
                 n_layers: int,
                 n_heads: int, 
                 d_model: int, 
                 max_len: int,
                 dropout_p: float = 0.5, 
                 dropout_p_attn: float = 0.5,
                 dropout_p_ff: float = 0.5,
                 dropout_p_addnorm: float = 0.5,
                 return_attns: bool = False
                ):
        super(Decoder, self).__init__()
        self.return_attns = return_attns
        
        self.layer_stack = nn.ModuleList([
            DecoderLayer(n_heads=n_heads,
                         d_model=d_model,
                         max_len=max_len,
                         dropout_p=dropout_p,
                         dropout_p_attn=dropout_p_attn,
                         dropout_p_ff=dropout_p_ff,
                         dropout_p_addnorm=dropout_p_addnorm,
                         return_attn=True)
            for _ in range(n_layers)
        ])
        
        
    def forward(self,
                x: torch.Tensor,  # torch.Size([batch_size, seq_length, word_dim])
                encoder_out: torch.Tensor,  # torch.Size([batch_size, seq_length, word_dim]) 
                pad_mask: Optional[torch.Tensor] = None,
                target_mask: Optional[torch.Tensor] = None
               ) -> torch.Tensor:
        dec_self_attn_list = []
        dec_source_target_attn_list = []
        
        for dec_layer in self.layer_stack:
            x, dec_self_attn, dec_source_target_attn = dec_layer(
                x, encoder_out, pad_mask, target_mask)
            dec_self_attn_list += [dec_self_attn] if self.return_attns else []
            dec_source_target_attn_list += [dec_source_target_attn] if self.return_attns else []
        
        if self.return_attns:
            return x, dec_self_attn_list, dec_source_target_attn_list
        
        else:
            return x

In [49]:
class Transformer(nn.Module):
    def __init__(self,
                 n_layers: int,
                 n_heads: int, 
                 d_model: int,
                 source_vocab_size: int,
                 target_vocab_size: int,
                 source_pad_idx: int,
                 target_pad_idx: int,
                 max_len: int,
                 dropout_p: float = 0.1, 
                 dropout_p_attn: float = 0.1,
                 dropout_p_ff: float = 0.1,
                 dropout_p_addnorm: float = 0.1
                ):
        super(Transformer, self).__init__()
        
        self.source_pad_idx = source_pad_idx
        self.target_pad_idx = target_pad_idx
        self.max_len = max_len
        
        self.source_token_embeding = nn.Embedding(
            num_embeddings=source_vocab_size,
            embedding_dim=d_model
        )
        
        self.source_positional_encoder = PositionalEncoding(
            d_model=d_model, dropout_p=dropout_p, max_len=max_len
        )
        
        self.target_token_embeding = nn.Embedding(
            num_embeddings=target_vocab_size,
            embedding_dim=d_model
        )
        
        self.target_positional_encoder = PositionalEncoding(
            d_model=d_model, dropout_p=dropout_p, max_len=max_len
        )
        
        self.encoder = Encoder(
            n_layers=n_layers, n_heads=n_heads, 
            d_model=d_model, max_len=max_len
        )
        
        self.decoder = Decoder(
            n_layers=n_layers, n_heads=n_heads, 
            d_model=d_model, max_len=max_len
        )
        
        self.fc = nn.Linear(
            in_features=d_model, out_features=target_vocab_size, bias=False)
    
    def get_pad_mask(self, x, pad_idx):
        return (x != pad_idx).unsqueeze(-2)
    
    def get_target_mask(self, target):
        return (torch.triu(torch.ones(self.max_len, self.max_len), 
                           diagonal=1) == 0).unsqueeze(0)
    
    def forward(self, source, target):
        source_pad_mask = self.get_pad_mask(source, self.source_pad_idx)
        source = self.source_token_embeding(source)
        source = self.source_positional_encoder(source)
        encoder_output = self.encoder(source, source_pad_mask)
        
        target_pad_mask = self.get_pad_mask(target, self.target_pad_idx)
        target_mask = self.get_target_mask(target)
        target = self.target_token_embeding(target)
        target = self.target_positional_encoder(target)
        decoder_out = self.decoder(target, encoder_output, 
                                   target_pad_mask, target_mask)
        
        return self.fc(decoder_out)
    
    def encode(self, source):
        source_pad_mask = self.get_pad_mask(source, self.source_pad_idx)
        source = self.source_token_embeding(source)
        source = self.source_positional_encoder(source)
        encoder_output = self.encoder(source, source_pad_mask)
        return encoder_output
    
    def decode(self, encoder_output, target):
        target_pad_mask = self.get_pad_mask(target, self.target_pad_idx)
        target_mask = self.get_target_mask(target)
        target = self.target_token_embeding(target)
        target = self.target_positional_encoder(target)
        decoder_out = self.decoder(target, encoder_output, 
                                   target_pad_mask, target_mask)
        return F.softmax(self.fc(decoder_out), dim=-1)

In [12]:
def calculate_loss(pred: torch.Tensor,  # torch.Size([batch, max_len, target_vocab_len]) 
                   t: torch.Tensor  # torch.Size([batch, max_len])
                  ) -> torch.Tensor:
    return F.cross_entropy(pred.transpose(1, 2), t)

In [13]:
def calculate_performance(pred: torch.Tensor, 
                          t: torch.Tensor, 
                          target_pad_idx: int
                         ) -> Tuple[torch.Tensor, int, int]:
    loss = calculate_loss(pred, t)
    
    _, prediction = torch.max(pred, dim=-1)
    n_corrects = prediction.eq(t).masked_select(t != target_pad_idx).sum().item()
    n_words = len(t.masked_select(t != target_pad_idx))
    
    if random.randint(1, 100) == 1:
        print('pred\n',
            [''.join([num2alpha[word_i.item()] 
                      for word_i in out_word 
                      if num2alpha[word_i.item()] != 'pad']) 
             for out_word in prediction])
        print('t\n',
            [''.join([num2alpha[word_i.item()] 
                      for word_i in out_word 
                      if num2alpha[word_i.item()] != 'pad']) 
             for out_word in t])
    
        print('result: ', n_corrects, n_words)
    
    return loss, n_corrects, n_words

In [14]:
class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''

    def __init__(self, optimizer, lr_mul, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.lr_mul = lr_mul
        self.d_model = d_model
        self.n_warmup_steps = n_warmup_steps
        self.n_steps = 0


    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()


    def zero_grad(self):
        "Zero out the gradients with the inner optimizer"
        self._optimizer.zero_grad()


    def _get_lr_scale(self):
        d_model = self.d_model
        n_steps, n_warmup_steps = self.n_steps, self.n_warmup_steps
        return (d_model ** -0.5) * min(n_steps ** (-0.5), 
                                       n_steps * n_warmup_steps ** (-1.5))


    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_steps += 1
        lr = self.lr_mul * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

In [15]:
class ShortenDataset(Dataset):
    def __init__(self, 
                 originals: list, 
                 shorts: list,
                 max_len: int
                ):
        self.pad_id = 0
        self.macron_id = 27
        self.start_id = 28
        self.end_id = 29
        
        transform = kakasi()
        self.max_len = max_len
             
        self.originals = [
            list(map(self.transform, 
                     list(''.join([item['kunrei'].replace('#', '-') 
                                   for item in transform.convert(text.replace('ー', '#'))]))
                    )) for text in originals
        ]
        
        self.shorts = [
            list(map(self.transform, 
                     list(''.join([item['kunrei'].replace('#', '-') 
                                   for item in transform.convert(text.replace('ー', '#'))]))
                    )) for text in shorts
        ]
        
        self.originals = list(map(self.padding, self.originals))
        self.shorts = list(map(self.shift_right, self.shorts))
        
        
    def __len__(self):
        return len(self.originals)
        
    def __getitem__(self, 
                    index: int
                   ) -> Tuple[torch.Tensor]:
        return torch.tensor(self.originals[index]), torch.tensor(self.shorts[index])
    
    def transform(self, alphabet: List[str]) -> List[str]:
        if alphabet.isalpha():
            return ord(alphabet) - ord('a') + 1
        elif alphabet == '-':
            return self.macron_id
        elif alphabet == "'":
            return ord('n') - ord('a') + 1
        
    def padding(self, alphabet_list: List[str]) -> List[str]:
        return alphabet_list + [self.pad_id for _ in range(self.max_len-len(alphabet_list))]
    
    def shift_right(self, alphabet_list: List[str]) -> List[str]:
        return [self.start_id] + alphabet_list + [self.end_id] + [self.pad_id for _ in range(self.max_len-len(alphabet_list)-2 + 1)]

In [16]:
def train_epoch(model,
                training_data,
                optimizer,
                opt
               ):
    model.train()
    
    total_loss, n_word_total, n_word_correct = 0, 0, 0 

    desc = '  - (Training)   '
    for source_batch, target_batch in tqdm(training_data, mininterval=2, desc=desc, leave=False):
        source_batch = source_batch.to(opt.device)
        target_batch = target_batch.to(opt.device)
        
        target_in_batch = target_batch[:,:-1]
        target_out_batch = target_batch[:,1:]
        
        # forward
        optimizer.zero_grad()
        pred = model(source_batch, target_in_batch)

        # backward and update parameters
        loss, n_correct, n_word = calculate_performance(
            pred, target_out_batch, opt.target_pad_idx) 
        loss.backward()
        optimizer.step_and_update_lr()

        # note keeping
        n_word_total += n_word
        n_word_correct += n_correct
        total_loss += loss.item()

    loss_per_word = total_loss / n_word_total
    accuracy = n_word_correct / n_word_total
    
    return loss_per_word, accuracy

In [17]:
def eval_epoch(model,
               validation_data,
               opt
               ):
    model.eval()
    
    total_loss, n_word_total, n_word_correct = 0, 0, 0 
    
    with torch.no_grad():
        desc = '  - (Validation)   '
        for source_batch, target_batch in tqdm(validation_data, mininterval=2, desc=desc, leave=False):
            source_batch = source_batch.to(opt.device)
            target_batch = target_batch.to(opt.device)

            target_in_batch = target_batch[:,:-1]
            target_out_batch = target_batch[:,1:]

            # forward
            pred = model(source_batch, target_in_batch)
            loss, n_correct, n_word = calculate_performance(
                pred, target_out_batch, opt.target_pad_idx)

            # note keeping
            n_word_total += n_word
            n_word_correct += n_correct
            total_loss += loss.item()

    loss_per_word = total_loss / n_word_total
    accuracy = n_word_correct / n_word_total
    
    return loss_per_word, accuracy

In [18]:
def train(model, training_data, validation_data, optimizer, opt):
    ''' Start training '''

    # Use tensorboard to plot curves, e.g. perplexity, accuracy, learning rate
    if opt.use_tb:
        print("[Info] Use Tensorboard")
        from torch.utils.tensorboard import SummaryWriter
        tb_writer = SummaryWriter(log_dir=os.path.join(opt.output_dir, 'tensorboard'))

    log_train_file = os.path.join(opt.output_dir, 'train.log')
    log_valid_file = os.path.join(opt.output_dir, 'valid.log')

    print('[Info] Training performance will be written to file: {} and {}'.format(
        log_train_file, log_valid_file))

    with open(log_train_file, 'w') as log_tf, open(log_valid_file, 'w') as log_vf:
        log_tf.write('epoch,loss,accuracy\n')
        log_vf.write('epoch,loss,accuracy\n')

    def print_performances(header, accu, start_time, lr):
        print('  - {header:12}  accuracy: {accu:3.3f} %, lr: {lr:8.5f}, '\
              'elapse: {elapse:3.3f} min'.format(
                  header=f"({header})",
                  accu=100*accu, elapse=(time.time()-start_time)/60, lr=lr))

    valid_losses = []
    for epoch_i in range(opt.epoch):
        print(f'[ Epoch{epoch_i} ]')

        start = time.time()
        train_loss, train_accu = train_epoch(
            model, training_data, optimizer, opt)
        # Current learning rate
        lr = optimizer._optimizer.param_groups[0]['lr']
        print_performances('Training', train_accu, start, lr)

        start = time.time()
        valid_loss, valid_accu = eval_epoch(model, validation_data, opt)
        print_performances('Validation', valid_accu, start, lr)

        valid_losses += [valid_loss]

        checkpoint = {'epoch': epoch_i, 'settings': opt, 'model': model.state_dict()}

        if opt.save_mode == 'all':
            model_name = f'model_accuracy_{100*valid_accu:3.3f}.chkpt'
            torch.save(checkpoint, model_name)
        elif opt.save_mode == 'best':
            model_name = 'model.chkpt'
            if valid_loss <= min(valid_losses):
                torch.save(checkpoint, os.path.join(opt.output_dir, model_name))
                print('    - [Info] The checkpoint file has been updated.')

        with open(log_train_file, 'a') as log_tf, open(log_valid_file, 'a') as log_vf:
            log_tf.write('{epoch},{loss: 8.5f},{accu:3.3f}\n'.format(
                epoch=epoch_i, loss=train_loss, accu=100*train_accu))
            log_vf.write('{epoch},{loss: 8.5f},{accu:3.3f}\n'.format(
                epoch=epoch_i, loss=valid_loss, accu=100*valid_accu))

        if opt.use_tb:
            tb_writer.add_scalars('accuracy', {'train': train_accu*100, 'val': valid_accu*100}, epoch_i)
            tb_writer.add_scalar('learning_rate', lr, epoch_i)

In [19]:
num2alpha = {0: 'pad',
             1: 'a',
             2: 'b',
             3: 'c',
             4: 'd',
             5: 'e',
             6: 'f',
             7: 'g',
             8: 'h',
             9: 'i',
             10: 'j',
             11: 'k',
             12: 'l',
             13: 'm',
             14: 'n',
             15: 'o',
             16: 'p',
             17: 'q',
             18: 'r',
             19: 's',
             20: 't',
             21: 'u',
             22: 'v',
             23: 'w',
             24: 'x',
             25: 'y',
             26: 'z',
             27: '-',
             28: 'start',
             29: 'end'
            }

```shell
!pip install tensorboard
```

In [20]:
parser = argparse.ArgumentParser()
parser.add_argument('-data_path', default=None)
parser.add_argument('-output_dir', type=str, default=None)
parser.add_argument('-b', '--batch_size', type=int, default=2048)
parser.add_argument('-epoch', type=int, default=10)
parser.add_argument('-d_model', type=int, default=512)
parser.add_argument('-n_heads', type=int, default=8)
parser.add_argument('-n_layers', type=int, default=6)
parser.add_argument('-warmup','--n_warmup_steps', type=int, default=4000)
parser.add_argument('-lr_mul', type=float, default=2.0)
parser.add_argument('-seed', type=int, default=0)
parser.add_argument('-save_mode', type=str, choices=['all', 'best'], default='best')
parser.add_argument('-use_tb', action='store_true')

opt = parser.parse_args(args=['-data_path', './data/train_manual.csv',
                              '-output_dir', './output',
                              '--batch_size', '20',
                              '-epoch','1000',
                              '-d_model', '64',
                              '-n_heads', '4',
                              '-n_layers', '1',
                              '-save_mode', 'best',
                              '-use_tb'])

# For reproducibility
if opt.seed is not None:
    torch.manual_seed(opt.seed)
    torch.backends.cudnn.benchmark = False
    # torch.set_deterministic(True)
    np.random.seed(opt.seed)
    random.seed(opt.seed)

df = pd.read_csv(opt.data_path, encoding='SHIFT-JIS')
df_train, df_val = train_test_split(df, test_size=0.1, random_state=opt.seed)
opt.max_len = 50

train_dataset = ShortenDataset(df_train['original'].tolist(), 
                               df_train['short'].tolist(),
                               opt.max_len)
val_dataset = ShortenDataset(df_val['original'].tolist(), 
                             df_val['short'].tolist(),
                             opt.max_len)

train_dataloader = DataLoader(train_dataset, 
                              batch_size=opt.batch_size, 
                              shuffle=True)
val_dataloader = DataLoader(val_dataset, 
                            batch_size=opt.batch_size, 
                            shuffle=False)

# アルファベット(26文字) + 特殊文字(pad, start, end, -) = 30文字
opt.source_vocab_size = 30
opt.target_vocab_size = 30
opt.source_pad_idx = 0
opt.target_pad_idx = 0

opt.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transformer = Transformer(n_layers=opt.n_layers,
                          n_heads=opt.n_heads,
                          d_model=opt.d_model,
                          source_vocab_size=opt.source_vocab_size,
                          target_vocab_size=opt.target_vocab_size,
                          source_pad_idx=opt.source_pad_idx,
                          target_pad_idx=opt.target_pad_idx,
                          max_len=opt.max_len
                         ).to(opt.device)

optimizer = ScheduledOptim(
    optim.Adam(transformer.parameters(), betas=(0.9, 0.98), eps=1e-09),
    opt.lr_mul, opt.d_model, opt.n_warmup_steps)

train(transformer, train_dataloader, val_dataloader, optimizer, opt)

[Info] Use Tensorboard
[Info] Training performance will be written to file: ./output/train.log and ./output/valid.log
[ Epoch0 ]


  - (Training)   :   0%|          | 0/29 [00:00<?, ?it/s]

  - (Training)    accuracy: 2.626 %, lr:  0.00003, elapse: 0.023 min


  - (Validation)   :   0%|          | 0/4 [00:00<?, ?it/s]

  - (Validation)  accuracy: 1.018 %, lr:  0.00003, elapse: 0.001 min
    - [Info] The checkpoint file has been updated.
[ Epoch1 ]


...

  - (Validation)  accuracy: 29.532 %, lr:  0.00147, elapse: 0.001 min
[ Epoch999 ]


  - (Training)   :   0%|          | 0/29 [00:00<?, ?it/s]

  - (Training)    accuracy: 30.615 %, lr:  0.00147, elapse: 0.033 min


  - (Validation)   :   0%|          | 0/4 [00:00<?, ?it/s]

  - (Validation)  accuracy: 28.717 %, lr:  0.00147, elapse: 0.001 min


# 改善点
- max_lenをbatchでのmax_lenに変更する -> pad_seqが使えるようになる
- hardlossではなく、softlossに変更する

# 推論

In [50]:
load_dict = torch.load('./output/model.chkpt')
model = load_dict['model']
opt = load_dict['settings']

In [51]:
transformer = Transformer(n_layers=opt.n_layers,
                          n_heads=opt.n_heads,
                          d_model=opt.d_model,
                          source_vocab_size=opt.source_vocab_size,
                          target_vocab_size=opt.target_vocab_size,
                          source_pad_idx=opt.source_pad_idx,
                          target_pad_idx=opt.target_pad_idx,
                          max_len=opt.max_len
                         ).to(opt.device)
transformer.load_state_dict(model)

<All keys matched successfully>

In [217]:
# source_word = ['ミスターチルドレン',
#                'ミスターチルドレン',
#                'ミスターチルドレン',
#                'ミスターチルドレン',
#                'ミスターチルドレン',
#                'ミスターチルドレン',
#                'ミスターチルドレン',
#                'ミスターチルドレン',
#                'ミスターチルドレン']
# target_word = ['', 
#                'm',
#                'mi',
#                'mis',
#                'misu',
#                'misut',
#                'misuti',
#                'misutir',
#                'misutiru'
#               ]

source_word = ['ミスターチルドレン', 'リモートコントローラー', 'コンビニエンスストア']
target_word = ['', '', '']

test_dataset = ShortenDataset(source_word, target_word, opt.max_len)
test_dataloader = DataLoader(test_dataset, batch_size=3, shuffle=False)
test_dataset[:]

(tensor([[13,  9, 19, 21, 20,  1, 27, 20,  9, 18, 21,  4, 15, 18,  5, 14,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [18,  9, 13, 15, 27, 20, 15, 11, 15, 14, 20, 15, 18, 15, 27, 18,  1, 27,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [11, 15, 14,  2,  9, 14,  9,  5, 14, 19, 21, 19, 21, 20, 15,  1,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]]),
 tensor([[28, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [28, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [218]:
transformer.eval()
max_len = opt.max_len
batch_size = 3
with torch.no_grad():
    for source_batch, target_batch in test_dataloader:
        print(target_batch)
        pred = transformer(source_batch, target_batch[:, :-1])
        print(torch.max(pred, dim=-1)[1])

tensor([[28, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [28, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [28, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
tensor([[11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0

In [ ]:
tensor([[11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21, 11,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21, 11,  1, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21, 11,  1, 29,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21, 11,  1, 29,  1, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

## greedy encoding

In [220]:
# 単語生成(greedy encoding)
transformer.eval()
max_len = opt.max_len
batch_size = 3
with torch.no_grad():
    for source_batch, _ in test_dataloader:
        generated = torch.empty((batch_size, max_len), 
                                dtype=torch.int64).fill_(0) # pad埋め
        generated[:, 0] = 28 # start
        print('******************')
        print(f'source\n{source_batch}')
        for i in range(1, max_len):
            # 対象の単語までだけを取り出し、pad埋めする
            target_generated = torch.cat(
                [generated[:, :i],
                 torch.empty((batch_size, max_len-i), 
                             dtype=torch.int64).fill_(0)], 
                dim=1
            )
            print(f'target\n{target_generated}')
            pred = transformer(source_batch.to(opt.device), 
                               target_generated.to(opt.device))
            max_value, next_words = torch.max(pred, dim=-1)
#             print(f'max_value\n{max_value}')
#             print(f'pred\n{pred}')
#             print(f'next_words\n{next_words}')
            generated[:, i] = next_words[:, i-1]
        print(f'output\n{generated}')
        break
generated

******************
source
tensor([[13,  9, 19, 21, 20,  1, 27, 20,  9, 18, 21,  4, 15, 18,  5, 14,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [18,  9, 13, 15, 27, 20, 15, 11, 15, 14, 20, 15, 18, 15, 27, 18,  1, 27,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11, 15, 14,  2,  9, 14,  9,  5, 14, 19, 21, 19, 21, 20, 15,  1,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
target
tensor([[28,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [28,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  

tensor([[28, 11,  1, 18,  1, 11,  1, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [28, 11,  1, 18,  1, 11,  1, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [28, 11,  1, 18,  1, 11,  1, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [479]:
# transformer.eval()
# with torch.no_grad():
#     for source_batch, target_batch in test_dataloader:
#         source_batch = source_batch.to(opt.device)
#         print(target_batch)
#         target_batch = target_batch.to(opt.device)

#         pred = transformer(source_batch, target_batch[:,:-1])

#         _, out_words = torch.max(pred, dim=-1)
#         print(out_words)
#         print([''.join([num2alpha[word_i.item()] for word_i in out_word if num2alpha[word_i.item()] != 'pad' and num2alpha[word_i.item()] != 'end']) for out_word in out_words])

## beam search

In [215]:
source_word = ['ミスターチルドレン', 'リモートコントローラー', 'コンビニエンスストア']
target_word = ['', '', '']
test_dataset = ShortenDataset(source_word, target_word, opt.max_len)
test_dataloader = DataLoader(test_dataset, batch_size=3, shuffle=False)

In [185]:
beam_size = 4

In [186]:
source, _ = next(iter(test_dataloader))
transformer.eval()
encoder_output = transformer.encode(source)
batch_size = source.size(0)
target_init = torch.full((batch_size, transformer.max_len), 
                         fill_value=0, dtype=torch.int64)
target_init[:, 0] = 28
decoder_output = transformer.decode(encoder_output, target_init)

In [191]:
encoder_output

tensor([[[-0.0313,  0.0931, -0.0655,  ...,  0.0035,  0.0604,  0.0489],
         [-0.0313,  0.0927, -0.0653,  ...,  0.0035,  0.0603,  0.0486],
         [-0.0313,  0.0928, -0.0653,  ...,  0.0035,  0.0603,  0.0486],
         ...,
         [-0.0313,  0.0928, -0.0653,  ...,  0.0035,  0.0603,  0.0486],
         [-0.0313,  0.0928, -0.0653,  ...,  0.0035,  0.0603,  0.0486],
         [-0.0314,  0.0929, -0.0653,  ...,  0.0035,  0.0603,  0.0486]],

        [[-0.0312,  0.0929, -0.0646,  ...,  0.0035,  0.0603,  0.0483],
         [-0.0312,  0.0927, -0.0647,  ...,  0.0035,  0.0603,  0.0483],
         [-0.0312,  0.0929, -0.0647,  ...,  0.0035,  0.0603,  0.0483],
         ...,
         [-0.0312,  0.0928, -0.0647,  ...,  0.0035,  0.0603,  0.0483],
         [-0.0312,  0.0928, -0.0647,  ...,  0.0035,  0.0603,  0.0483],
         [-0.0313,  0.0927, -0.0646,  ...,  0.0035,  0.0603,  0.0481]],

        [[-0.0314,  0.0932, -0.0656,  ...,  0.0035,  0.0604,  0.0490],
         [-0.0314,  0.0928, -0.0653,  ...,  0

In [192]:
decoder_output.shape, decoder_output

(torch.Size([3, 50, 30]),
 tensor([[[3.6391e-04, 4.3200e-02, 5.6538e-02,  ..., 8.1605e-03,
           4.2743e-07, 4.9940e-03],
          [1.0000e+00, 2.2716e-08, 2.0247e-08,  ..., 2.0468e-08,
           4.8579e-13, 5.6915e-08],
          [1.0000e+00, 2.2445e-08, 2.0957e-08,  ..., 2.0698e-08,
           5.1436e-13, 5.7531e-08],
          ...,
          [1.0000e+00, 1.9180e-08, 1.8877e-08,  ..., 1.9325e-08,
           6.6022e-13, 6.0394e-08],
          [1.0000e+00, 1.9063e-08, 1.8392e-08,  ..., 1.8807e-08,
           6.7355e-13, 6.0597e-08],
          [1.0000e+00, 1.8840e-08, 1.7726e-08,  ..., 1.8489e-08,
           6.7437e-13, 6.1679e-08]],
 
         [[3.6403e-04, 4.3196e-02, 5.6541e-02,  ..., 8.1612e-03,
           4.2731e-07, 4.9963e-03],
          [1.0000e+00, 2.2696e-08, 2.0233e-08,  ..., 2.0465e-08,
           4.8565e-13, 5.6936e-08],
          [1.0000e+00, 2.2423e-08, 2.0940e-08,  ..., 2.0691e-08,
           5.1440e-13, 5.7540e-08],
          ...,
          [1.0000e+00, 1.9166e-0

一番初めにくる単語の中でtopkまで取得しておく

In [193]:
decoder_output[:, 0, :]

tensor([[3.6391e-04, 4.3200e-02, 5.6538e-02, 4.2765e-07, 4.6877e-02, 3.4233e-02,
         2.8358e-02, 3.4660e-02, 4.2975e-02, 3.4243e-02, 2.5181e-03, 1.2818e-01,
         4.2823e-07, 8.1087e-02, 4.5713e-02, 2.5512e-02, 6.9491e-02, 4.2754e-07,
         4.8418e-02, 1.1876e-01, 9.4422e-02, 8.3209e-03, 1.1599e-03, 1.0189e-02,
         4.2831e-07, 1.4466e-02, 1.7158e-02, 8.1605e-03, 4.2743e-07, 4.9940e-03],
        [3.6403e-04, 4.3196e-02, 5.6541e-02, 4.2753e-07, 4.6880e-02, 3.4230e-02,
         2.8358e-02, 3.4662e-02, 4.2977e-02, 3.4241e-02, 2.5184e-03, 1.2818e-01,
         4.2811e-07, 8.1087e-02, 4.5711e-02, 2.5510e-02, 6.9493e-02, 4.2742e-07,
         4.8416e-02, 1.1876e-01, 9.4423e-02, 8.3210e-03, 1.1599e-03, 1.0189e-02,
         4.2819e-07, 1.4465e-02, 1.7158e-02, 8.1612e-03, 4.2731e-07, 4.9963e-03],
        [3.6390e-04, 4.3200e-02, 5.6538e-02, 4.2767e-07, 4.6877e-02, 3.4233e-02,
         2.8358e-02, 3.4660e-02, 4.2975e-02, 3.4243e-02, 2.5181e-03, 1.2817e-01,
         4.2825e-07, 8.108

In [194]:
top_k_probs, top_k_idx = decoder_output[:, 0, :].topk(beam_size)
top_k_probs, top_k_idx

(tensor([[0.1282, 0.1188, 0.0944, 0.0811],
         [0.1282, 0.1188, 0.0944, 0.0811],
         [0.1282, 0.1188, 0.0944, 0.0811]], grad_fn=<TopkBackward>),
 tensor([[11, 19, 20, 13],
         [11, 19, 20, 13],
         [11, 19, 20, 13]]))

In [195]:
scores = torch.log(top_k_probs).contiguous().view(batch_size, beam_size)
scores

tensor([[-2.0544, -2.1306, -2.3600, -2.5122],
        [-2.0543, -2.1306, -2.3600, -2.5122],
        [-2.0544, -2.1306, -2.3600, -2.5122]], grad_fn=<ViewBackward>)

In [196]:
blank_sequences = torch.full((beam_size*batch_size, opt.max_len), 0)
blank_sequences[:, 0] = 28
generated_sequences = blank_sequences.clone().detach()
generated_sequences[:, 1] = top_k_idx.contiguous().view(beam_size*batch_size)
generated_sequences.shape, generated_sequences

(torch.Size([12, 50]),
 tensor([[28, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [28, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [28, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [28, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [28, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [197]:
encoder_output.shape

torch.Size([3, 50, 64])

In [198]:
encoder_output = encoder_output.repeat(beam_size, 1, 1)
encoder_output.shape

torch.Size([12, 50, 64])

In [199]:
decoder_output = transformer.decode(encoder_output, generated_sequences)
decoder_output.shape, decoder_output

(torch.Size([12, 50, 30]),
 tensor([[[3.6394e-04, 4.3199e-02, 5.6539e-02,  ..., 8.1607e-03,
           4.2740e-07, 4.9947e-03],
          [2.8259e-06, 2.4519e-01, 1.0703e-04,  ..., 1.1448e-05,
           5.0355e-12, 3.6196e-06],
          [1.0000e+00, 2.2199e-08, 2.0833e-08,  ..., 2.0689e-08,
           5.1496e-13, 5.7677e-08],
          ...,
          [1.0000e+00, 1.8444e-08, 1.8563e-08,  ..., 1.9468e-08,
           6.5828e-13, 6.1046e-08],
          [1.0000e+00, 1.8298e-08, 1.8249e-08,  ..., 1.9251e-08,
           6.3876e-13, 6.2372e-08],
          [1.0000e+00, 1.8041e-08, 1.7745e-08,  ..., 1.9197e-08,
           6.0945e-13, 6.4311e-08]],
 
         [[3.6403e-04, 4.3196e-02, 5.6541e-02,  ..., 8.1612e-03,
           4.2731e-07, 4.9964e-03],
          [5.8562e-06, 1.9383e-01, 1.1859e-04,  ..., 2.6144e-05,
           2.2748e-12, 3.5768e-05],
          [1.0000e+00, 2.2196e-08, 2.0813e-08,  ..., 2.0678e-08,
           5.1404e-13, 5.7755e-08],
          ...,
          [1.0000e+00, 2.8333e-

２番目にくる単語の各topk単語を取得する

In [200]:
top_k2_probs, top_k2_idx = decoder_output[:, 1, :].topk(beam_size)
top_k2_probs, top_k2_idx

(tensor([[0.2452, 0.2051, 0.1721, 0.1630],
         [0.4344, 0.1938, 0.1218, 0.1061],
         [0.2610, 0.2118, 0.1871, 0.1555],
         [0.3135, 0.1985, 0.1917, 0.1677],
         [0.2452, 0.2051, 0.1721, 0.1630],
         [0.4344, 0.1938, 0.1219, 0.1061],
         [0.2610, 0.2118, 0.1871, 0.1555],
         [0.3135, 0.1985, 0.1917, 0.1677],
         [0.2452, 0.2051, 0.1721, 0.1630],
         [0.4344, 0.1938, 0.1219, 0.1061],
         [0.2610, 0.2118, 0.1871, 0.1554],
         [0.3135, 0.1985, 0.1917, 0.1677]], grad_fn=<TopkBackward>),
 tensor([[ 1, 15,  5, 21],
         [21,  1, 15,  9],
         [ 1, 15,  5,  9],
         [ 1,  5, 15,  9],
         [ 1, 15,  5, 21],
         [21,  1, 15,  9],
         [ 1, 15,  5,  9],
         [ 1,  5, 15,  9],
         [ 1, 15,  5, 21],
         [21,  1, 15,  9],
         [ 1, 15,  5,  9],
         [ 1,  5, 15,  9]]))

In [201]:
scores_2 = torch.log(top_k2_probs).contiguous().view(batch_size, 
                                                     beam_size, 
                                                     -1)
scores_2

tensor([[[-1.4057, -1.5841, -1.7599, -1.8143],
         [-0.8337, -1.6408, -2.1050, -2.2434],
         [-1.3433, -1.5523, -1.6763, -1.8614],
         [-1.1598, -1.6168, -1.6518, -1.7854]],

        [[-1.4058, -1.5841, -1.7599, -1.8142],
         [-0.8339, -1.6407, -2.1047, -2.2435],
         [-1.3433, -1.5523, -1.6763, -1.8614],
         [-1.1599, -1.6168, -1.6517, -1.7854]],

        [[-1.4057, -1.5841, -1.7599, -1.8143],
         [-0.8339, -1.6407, -2.1047, -2.2434],
         [-1.3434, -1.5523, -1.6763, -1.8615],
         [-1.1598, -1.6168, -1.6518, -1.7854]]], grad_fn=<ViewBackward>)

In [202]:
scores, scores.unsqueeze(1).expand_as(scores_2)

(tensor([[-2.0544, -2.1306, -2.3600, -2.5122],
         [-2.0543, -2.1306, -2.3600, -2.5122],
         [-2.0544, -2.1306, -2.3600, -2.5122]], grad_fn=<ViewBackward>),
 tensor([[[-2.0544, -2.1306, -2.3600, -2.5122],
          [-2.0544, -2.1306, -2.3600, -2.5122],
          [-2.0544, -2.1306, -2.3600, -2.5122],
          [-2.0544, -2.1306, -2.3600, -2.5122]],
 
         [[-2.0543, -2.1306, -2.3600, -2.5122],
          [-2.0543, -2.1306, -2.3600, -2.5122],
          [-2.0543, -2.1306, -2.3600, -2.5122],
          [-2.0543, -2.1306, -2.3600, -2.5122]],
 
         [[-2.0544, -2.1306, -2.3600, -2.5122],
          [-2.0544, -2.1306, -2.3600, -2.5122],
          [-2.0544, -2.1306, -2.3600, -2.5122],
          [-2.0544, -2.1306, -2.3600, -2.5122]]], grad_fn=<ExpandBackward>))

In [203]:
scores = scores.unsqueeze(1).expand_as(scores_2) + scores_2
scores.shape, scores

(torch.Size([3, 4, 4]),
 tensor([[[-3.4601, -3.7147, -4.1199, -4.3265],
          [-2.8880, -3.7714, -4.4650, -4.7556],
          [-3.3977, -3.6829, -4.0363, -4.3736],
          [-3.2142, -3.7474, -4.0118, -4.2976]],
 
         [[-3.4601, -3.7148, -4.1199, -4.3264],
          [-2.8882, -3.7713, -4.4647, -4.7557],
          [-3.3977, -3.6830, -4.0363, -4.3736],
          [-3.2142, -3.7475, -4.0117, -4.2977]],
 
         [[-3.4601, -3.7147, -4.1199, -4.3265],
          [-2.8882, -3.7713, -4.4647, -4.7557],
          [-3.3978, -3.6829, -4.0363, -4.3737],
          [-3.2142, -3.7474, -4.0118, -4.2976]]], grad_fn=<AddBackward0>))

In [204]:
scores, top_k_idx_in_k2 = scores.contiguous().view(
    batch_size, -1).topk(beam_size)
scores, top_k_idx_in_k2

(tensor([[-2.8880, -3.2142, -3.3977, -3.4601],
         [-2.8882, -3.2142, -3.3977, -3.4601],
         [-2.8882, -3.2142, -3.3978, -3.4601]], grad_fn=<TopkBackward>),
 tensor([[ 4, 12,  8,  0],
         [ 4, 12,  8,  0],
         [ 4, 12,  8,  0]]))

In [205]:
top_k2_idx_row = top_k_idx_in_k2 // beam_size
top_k2_idx_col = top_k_idx_in_k2 % beam_size
top_k2_idx_row, top_k2_idx_col

(tensor([[1, 3, 2, 0],
         [1, 3, 2, 0],
         [1, 3, 2, 0]]),
 tensor([[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]))

In [206]:
top_k2_idx

tensor([[ 1, 15,  5, 21],
        [21,  1, 15,  9],
        [ 1, 15,  5,  9],
        [ 1,  5, 15,  9],
        [ 1, 15,  5, 21],
        [21,  1, 15,  9],
        [ 1, 15,  5,  9],
        [ 1,  5, 15,  9],
        [ 1, 15,  5, 21],
        [21,  1, 15,  9],
        [ 1, 15,  5,  9],
        [ 1,  5, 15,  9]])

In [207]:
top_k_idx = top_k2_idx[top_k2_idx_row, top_k2_idx_col]
top_k_idx

tensor([[21,  1,  1,  1],
        [21,  1,  1,  1],
        [21,  1,  1,  1]])

In [208]:
generated_sequences[:, :2]

tensor([[28, 11],
        [28, 19],
        [28, 20],
        [28, 13],
        [28, 11],
        [28, 19],
        [28, 20],
        [28, 13],
        [28, 11],
        [28, 19],
        [28, 20],
        [28, 13]])

In [209]:
generated_sequences[top_k2_idx_row.view(-1), :2]

tensor([[28, 19],
        [28, 13],
        [28, 20],
        [28, 11],
        [28, 19],
        [28, 13],
        [28, 20],
        [28, 11],
        [28, 19],
        [28, 13],
        [28, 20],
        [28, 11]])

In [210]:
# topkに該当するもので、それまでの単語を更新
generated_sequences[:, :2] = generated_sequences[top_k2_idx_row.view(-1), :2]
generated_sequences[:, 2] = top_k_idx.view(-1)
generated_sequences[:, :5]

tensor([[28, 19, 21,  0,  0],
        [28, 13,  1,  0,  0],
        [28, 20,  1,  0,  0],
        [28, 11,  1,  0,  0],
        [28, 19, 21,  0,  0],
        [28, 13,  1,  0,  0],
        [28, 20,  1,  0,  0],
        [28, 11,  1,  0,  0],
        [28, 19, 21,  0,  0],
        [28, 13,  1,  0,  0],
        [28, 20,  1,  0,  0],
        [28, 11,  1,  0,  0]])

上記をfor文で繰り返す

In [212]:
source, _ = next(iter(test_dataloader))
transformer.eval()
encoder_output = transformer.encode(source)
batch_size = source.size(0)
target_init = torch.full((batch_size, transformer.max_len), fill_value=0, dtype=torch.int64)
target_init[:, 0] = 28
decoder_output = transformer.decode(encoder_output, target_init)

top_k_probs, top_k_idx = decoder_output[:, 0, :].topk(beam_size)
scores = torch.log(top_k_probs).contiguous().view(batch_size, beam_size)

blank_sequences = torch.full((beam_size*batch_size, opt.max_len), 0)
blank_sequences[:, 0] = 28
generated_sequences = blank_sequences.clone().detach()
generated_sequences[:, 1] = top_k_idx.contiguous().view(beam_size*batch_size)

encoder_output = encoder_output.repeat(beam_size, 1, 1)
decoder_output = transformer.decode(encoder_output, generated_sequences)

for step in range(2, opt.max_len):
    top_k2_probs, top_k2_idx = decoder_output[:, 1, :].topk(beam_size)
    scores_2 = torch.log(top_k2_probs).contiguous().view(batch_size, 
                                                         beam_size, 
                                                         beam_size)
    scores = scores.unsqueeze(1).expand_as(scores_2) + scores_2
    scores, top_k_idx_in_k2 = scores.contiguous().view(
        batch_size, -1).topk(beam_size)
    print(scores, top_k_idx_in_k2)
    top_k2_idx_row = top_k_idx_in_k2 // beam_size
    top_k2_idx_col = top_k_idx_in_k2 % beam_size
    top_k_idx = top_k2_idx[top_k2_idx_row, top_k2_idx_col]
    generated_sequences[:, :step] = generated_sequences[
        top_k2_idx_row.view(-1), :step]
    generated_sequences[:, step] = top_k_idx.view(-1)
    print(generated_sequences)
    print('********************')
generated_sequences

tensor([[-2.8880, -3.2142, -3.3977, -3.4601],
        [-2.8882, -3.2142, -3.3977, -3.4601],
        [-2.8882, -3.2142, -3.3978, -3.4601]], grad_fn=<TopkBackward>) tensor([[ 4, 12,  8,  0],
        [ 4, 12,  8,  0],
        [ 4, 12,  8,  0]])
tensor([[28, 19, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [28, 13,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [28, 20,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [28, 11,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  

tensor([[28, 11,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,
          1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,
          1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21],
        [28, 19, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,
          1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,
          1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1],
        [28, 20,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [28, 13,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1,
         21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1,
         21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1],
        [28, 11,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,
       

In [ ]:
tensor([[11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21, 11,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21, 11,  1, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21, 11,  1, 29,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11,  1, 18, 21, 11,  1, 29,  1, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [104]:
scores

tensor([[-42.0793, -42.4053, -42.5888, -42.6512]], grad_fn=<TopkBackward>)

In [105]:
_, max_idx = scores.max(dim=-1)
generated_sequences[max_idx]

tensor([[28, 11,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,
          1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,
          1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21,  1,  1, 21]])

In [ ]:
class Translator(nn.Module):
    def __init__(self, model):
        super(Translator, self).__init__()

        

In [68]:
num2alpha = {0: 'pad',
             1: 'a',
             2: 'b',
             3: 'c',
             4: 'd',
             5: 'e',
             6: 'f',
             7: 'g',
             8: 'h',
             9: 'i',
             10: 'j',
             11: 'k',
             12: 'l',
             13: 'm',
             14: 'n',
             15: 'o',
             16: 'p',
             17: 'q',
             18: 'r',
             19: 's',
             20: 't',
             21: 'u',
             22: 'v',
             23: 'w',
             24: 'x',
             25: 'y',
             26: 'z',
             27: '-',
             28: 'start',
             29: 'end'
            }

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z'}

In [695]:
df = pd.read_csv(opt.data_path, encoding='SHIFT-JIS')
df_train, df_val = train_test_split(df, test_size=0.2, random_state=opt.seed)
df_train

,short,original
245,バイト,アルバイト
45,オフレコ,オフザレコード
6,アジカン,アジアンカンフージェネレーション
205,テレわか,テレビ和歌山
178,着メロ,着信メロディ
...,...,...
277,ファザコン,ファザーコンプレックス
9,アニソン,アニメソング
359,メンコン,メンデルスゾーンヴァイオリンコンチェルト
192,デパガ,デパートガール


In [67]:
'リモコン' in df_train['short'].tolist()

True

In [68]:
df_val['short']

65     キムタク
132    スクエニ
74      キンキ
78     クソゲー
37     エムステ
       ... 
268     バンプ
234    ニンドリ
145    ストツー
164    セラムン
124     シャナ
Name: short, Length: 80, dtype: object